# Clear memory and set working directory

In [ ]:
rm(list = ls()) # clear the workspace
gc() # clear the memory
setwd("~/Myra_spns_snRNAseq_2023/") # set the working directory
getwd() # check the working directory

In [2]:
options(future.globals.maxSize = 40 * 1024 ^ 3) # for 20 Gb RAM # set the maximum size of the global environment for the future package for parallel computing
options(future.seed=TRUE) # set seed for reproducibility

# Load libraries

In [ ]:
# load the libraries
library(tidyverse)
library(Seurat)
library(scDblFinder)
library(patchwork)
library(SingleCellExperiment)
library(harmony)
library(BiocParallel)
library(repr)
library(Libra)
library(clusterProfiler)
library(dittoSeq)
library(ggrepel)
library(dittoSeq)
library(paletteer)
library(ggpattern) 
library(future)
library(liana)
library(viridis)
library(enrichR) 
library(ggsankey)
library(paletteer)
options(repr.plot.width=16, repr.plot.height=10) # change the size of the plots
RNGkind("L'Ecuyer-CMRG") # set the random number generator
set.seed(1) # set the seed for reproducibility
bp <- MulticoreParam( RNGseed=1234) # set the seed for reproducibility for BiocParallel

plan("multicore", workers = 4) # set the number of cores for parallel computing

options(ggrepel.max.overlaps = 20) # set the maximum number of overlaps for ggrepel

# Load data

## sib_1

In [ ]:
# load the data for the first sample
sib_1_seurat<- Read10X('Cellranger_outs/sib_1_cellranger/outs/filtered_feature_bc_matrix/')
sib_1_seurat <- CreateSeuratObject(sib_1_seurat,project = "sib_1")
sib_1_seurat

In [6]:
# Calculate mitochondrial percentage
sib_1_seurat[["percent.mt"]] <- PercentageFeatureSet(sib_1_seurat, pattern = "^mt-")



In [ ]:
sib_1_seurat@meta.data %>% head

In [ ]:
median(sib_1_seurat@meta.data$percent.mt)
median(sib_1_seurat@meta.data$nCount_RNA)
median(sib_1_seurat@meta.data$nFeature_RNA)
dim(sib_1_seurat@meta.data)

In [ ]:
# plot the relation of the percentage mitochondria and the number of UMIs for the first sample
plot1 <- FeatureScatter(sib_1_seurat, feature1 = "nCount_RNA", feature2 = "percent.mt")
# plot the percentage of relation of the number of genes and the number of UMIs for the first sample
plot2 <- FeatureScatter(sib_1_seurat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

options(repr.plot.width=12, repr.plot.height=12)
plot1 + plot2
# plot the percentage of mitochondrial genes, number of genes and the number of UMIs for the first sample
VlnPlot(sib_1_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)

In [ ]:
# filter the cells for the first sample based on the number of genes, the number of UMIs, the percentage of mitochondrial genes and the doublet scores
sib_1_seurat <- subset(sib_1_seurat, subset = nFeature_RNA > 200 & nFeature_RNA < 5000 & 
                    percent.mt < 20 & nCount_RNA <10000)
sib_1_seurat

In [ ]:
# remove the mitochondrial genes
counts <- GetAssayData(sib_1_seurat, assay = "RNA")
non_mt_genes <- rownames(counts)[!grepl("^mt-", rownames(counts))]
sib_1_seurat <- subset(sib_1_seurat, features = non_mt_genes)
sib_1_seurat
sib_1_seurat[["percent.mt_after_removal"]] <- PercentageFeatureSet(sib_1_seurat, pattern = "^mt-") # calculate the percentage of mitochondrial genes after removal
VlnPlot(sib_1_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt", "percent.mt_after_removal"), ncol = 3)

### Calculate the doublet cells using scDblFinder

In [12]:
# normalize the data for the first sample using the SCTransform method
sib_1_seurat <- SCTransform(sib_1_seurat, verbose = FALSE,vars.to.regress = "percent.mt") %>%
    RunPCA(npcs = 50, verbose = FALSE)

In [ ]:
# convert the Seurat object to SingleCellExperiment object to be used in the scDblFinder package
sce_sib_1 <- as.SingleCellExperiment(sib_1_seurat, slot="counts", reducedDims = "pca")
sce_sib_1

In [ ]:
# convert the Seurat object to SingleCellExperiment object to be used in the scDblFinder package
sce_sib_1 <- scDblFinder(sce_sib_1, BPPARAM = bp)


In [ ]:
# sce_sib_1 %>% str
sce_sib_1$scDblFinder.score %>% head


In [ ]:
# port the resulting scores back to the Seurat object:
sib_1_seurat$scDblFinder.score <- sce_sib_1$scDblFinder.score
sib_1_seurat$scDblFinder.class <- sce_sib_1$scDblFinder.class
sib_1_seurat@meta.data %>% head

In [ ]:
sib_1_seurat <- subset(sib_1_seurat, subset = scDblFinder.class == "singlet")
sib_1_seurat

## Sib_2

In [ ]:
# load the data for the first sample
sib_2_seurat<- Read10X('Cellranger_outs/sib_2_cellranger/outs/filtered_feature_bc_matrix/')
sib_2_seurat <- CreateSeuratObject(sib_2_seurat,project = "sib_2")
sib_2_seurat
sib_2_seurat[["percent.mt"]] <- PercentageFeatureSet(sib_2_seurat, pattern = "^mt-")


In [ ]:
median(sib_2_seurat@meta.data$percent.mt)
median(sib_2_seurat@meta.data$nCount_RNA)
median(sib_2_seurat@meta.data$nFeature_RNA)
dim(sib_2_seurat@meta.data)

In [ ]:
# plot the relation of the percentage mitochondria and the number of UMIs for the first sample
plot1 <- FeatureScatter(sib_2_seurat, feature1 = "nCount_RNA", feature2 = "percent.mt")
# plot the percentage of relation of the number of genes and the number of UMIs for the first sample
plot2 <- FeatureScatter(sib_2_seurat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

options(repr.plot.width=12, repr.plot.height=12)
plot1 + plot2
# plot the percentage of mitochondrial genes, number of genes and the number of UMIs for the first sample
VlnPlot(sib_2_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)

In [ ]:
# filter the cells for the first sample based on the number of genes, the number of UMIs, the percentage of mitochondrial genes and the doublet scores
sib_2_seurat <- subset(sib_2_seurat, subset = nFeature_RNA > 200 & nFeature_RNA < 5000 & 
                    percent.mt < 20 & nCount_RNA <10000)
sib_2_seurat

In [ ]:
counts <- GetAssayData(sib_2_seurat, assay = "RNA")
non_mt_genes <- rownames(counts)[!grepl("^mt-", rownames(counts))]
sib_2_seurat <- subset(sib_2_seurat, features = non_mt_genes)
sib_2_seurat
sib_2_seurat[["percent.mt_after_removal"]] <- PercentageFeatureSet(sib_2_seurat, pattern = "^mt-") # calculate the percentage of mitochondrial genes after removal
VlnPlot(sib_2_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt", "percent.mt_after_removal"), ncol = 3)

In [23]:
sib_2_seurat <- SCTransform(sib_2_seurat, verbose = FALSE,vars.to.regress = "percent.mt") %>%
    RunPCA(npcs = 50, verbose = FALSE)

### Calculate the doublet cells using scDblFinder

In [ ]:
sce_sib_2 <- as.SingleCellExperiment(sib_2_seurat, slot="counts", reducedDims = "pca")
sce_sib_2

In [ ]:
sce_sib_2 <- scDblFinder(sce_sib_2, BPPARAM = bp)
# port the resulting scores back to the Seurat object:
sib_2_seurat$scDblFinder.score <- sce_sib_2$scDblFinder.score
sib_2_seurat$scDblFinder.class <- sce_sib_2$scDblFinder.class
sib_2_seurat@meta.data %>% head

In [ ]:
sib_2_seurat <- subset(sib_2_seurat, subset = scDblFinder.class == "singlet")
sib_2_seurat

## mut_1

In [ ]:
# load the data for the first sample
mut_1_seurat<- Read10X('Cellranger_outs/mut_1_cellranger/outs/filtered_feature_bc_matrix/')
mut_1_seurat <- CreateSeuratObject(mut_1_seurat,project = "mut_1")
mut_1_seurat
mut_1_seurat[["percent.mt"]] <- PercentageFeatureSet(mut_1_seurat, pattern = "^mt-")



In [ ]:
median(mut_1_seurat@meta.data$percent.mt)
median(mut_1_seurat@meta.data$nCount_RNA)
median(mut_1_seurat@meta.data$nFeature_RNA)
dim(mut_1_seurat@meta.data)

In [ ]:
# plot the relation of the percentage mitochondria and the number of UMIs for the first sample
plot1 <- FeatureScatter(mut_1_seurat, feature1 = "nCount_RNA", feature2 = "percent.mt")
# plot the percentage of relation of the number of genes and the number of UMIs for the first sample
plot2 <- FeatureScatter(mut_1_seurat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

options(repr.plot.width=12, repr.plot.height=12)
plot1 + plot2
# plot the percentage of mitochondrial genes, number of genes and the number of UMIs for the first sample
VlnPlot(mut_1_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)

In [ ]:
# filter the cells for the first sample based on the number of genes, the number of UMIs, the percentage of mitochondrial genes and the doublet scores
mut_1_seurat <- subset(mut_1_seurat, subset = nFeature_RNA > 200 & nFeature_RNA < 5000 & 
                    percent.mt < 20 & nCount_RNA <10000)
mut_1_seurat

In [ ]:
counts <- GetAssayData(mut_1_seurat, assay = "RNA")
non_mt_genes <- rownames(counts)[!grepl("^mt-", rownames(counts))]
mut_1_seurat <- subset(mut_1_seurat, features = non_mt_genes)
mut_1_seurat
mut_1_seurat[["percent.mt_after_removal"]] <- PercentageFeatureSet(mut_1_seurat, pattern = "^mt-") # calculate the percentage of mitochondrial genes after removal
VlnPlot(mut_1_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt", "percent.mt_after_removal"), ncol = 3)

In [32]:
mut_1_seurat <- SCTransform(mut_1_seurat, verbose = FALSE,vars.to.regress = "percent.mt") %>%
    RunPCA(npcs = 50, verbose = FALSE)

### Calculate the doublet cells using scDblFinder

In [ ]:
sce_mut_1 <- as.SingleCellExperiment(mut_1_seurat, slot="counts", reducedDims = "pca")
sce_mut_1

In [ ]:
sce_mut_1 <- scDblFinder(sce_mut_1, BPPARAM = bp)


In [ ]:
# port the resulting scores back to the Seurat object:
mut_1_seurat$scDblFinder.score <- sce_mut_1$scDblFinder.score
mut_1_seurat$scDblFinder.class <- sce_mut_1$scDblFinder.class
mut_1_seurat@meta.data %>% head

In [ ]:
mut_1_seurat <- subset(mut_1_seurat, subset = scDblFinder.class == "singlet")
mut_1_seurat

## mut_2

In [ ]:
# load the data for the first sample
mut_2_seurat<- Read10X('Cellranger_outs/mut_2_cellranger/outs/filtered_feature_bc_matrix/')
mut_2_seurat <- CreateSeuratObject(mut_2_seurat,project = "mut_2")
mut_2_seurat
mut_2_seurat[["percent.mt"]] <- PercentageFeatureSet(mut_2_seurat, pattern = "^mt-")



In [ ]:
median(mut_2_seurat@meta.data$percent.mt)
median(mut_2_seurat@meta.data$nCount_RNA)
median(mut_2_seurat@meta.data$nFeature_RNA)
dim(mut_2_seurat@meta.data)

In [ ]:
# plot the relation of the percentage mitochondria and the number of UMIs for the first sample
plot1 <- FeatureScatter(mut_2_seurat, feature1 = "nCount_RNA", feature2 = "percent.mt")
# plot the percentage of relation of the number of genes and the number of UMIs for the first sample
plot2 <- FeatureScatter(mut_2_seurat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

options(repr.plot.width=12, repr.plot.height=12)
plot1 + plot2
# plot the percentage of mitochondrial genes, number of genes and the number of UMIs for the first sample
VlnPlot(mut_2_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)

In [ ]:
# filter the cells for the first sample based on the number of genes, the number of UMIs, the percentage of mitochondrial genes and the doublet scores
mut_2_seurat <- subset(mut_2_seurat, subset = nFeature_RNA > 200 & nFeature_RNA < 5000 & 
                    percent.mt < 20 & nCount_RNA <10000)
mut_2_seurat

In [ ]:
counts <- GetAssayData(mut_2_seurat, assay = "RNA")
non_mt_genes <- rownames(counts)[!grepl("^mt-", rownames(counts))]
mut_2_seurat <- subset(mut_2_seurat, features = non_mt_genes)
mut_2_seurat
mut_2_seurat[["percent.mt_after_removal"]] <- PercentageFeatureSet(mut_2_seurat, pattern = "^mt-") # calculate the percentage of mitochondrial genes after removal
VlnPlot(mut_2_seurat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt", "percent.mt_after_removal"), ncol = 3)

In [44]:
mut_2_seurat <- SCTransform(mut_2_seurat, verbose = FALSE,vars.to.regress = "percent.mt") %>%
    RunPCA(npcs = 50, verbose = FALSE)

### Calculate the doublet cells using scDblFinder

In [ ]:
sce_mut_2 <- as.SingleCellExperiment(mut_2_seurat, slot="counts", reducedDims = "pca")
sce_mut_2

In [ ]:
sce_mut_2 <- scDblFinder(sce_mut_2, BPPARAM = bp)


In [ ]:
# port the resulting scores back to the Seurat object:
mut_2_seurat$scDblFinder.score <- sce_mut_2$scDblFinder.score
mut_2_seurat$scDblFinder.class <- sce_mut_2$scDblFinder.class
mut_2_seurat@meta.data %>% head

In [ ]:
mut_2_seurat <- subset(mut_2_seurat, subset = scDblFinder.class == "singlet")
mut_2_seurat

# Batch correction using Seurat

## combine all datasets

According to benchmarking- LIGER, HARMONY and Seurat are all good. https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1850-9

In [ ]:
all_samples_list <- list(sib_1_seurat, sib_2_seurat, mut_1_seurat, mut_2_seurat) # create a list of all samples
all_samples_list 

In [ ]:
names(all_samples_list) <- c("sib_1", "sib_2", "mut_1", "mut_2") # name the samples
all_samples_list

In [54]:
all_samples_features <- SelectIntegrationFeatures(object.list = all_samples_list, nfeatures = 3000) # select the features for integration

In [56]:
all_samples_list <- PrepSCTIntegration(object.list = all_samples_list, anchor.features = all_samples_features) # prepare the samples for integration

In [ ]:
# Integrate the samples according to seurat pipeline
cell_anchors <- FindIntegrationAnchors(object.list = all_samples_list, normalization.method = "SCT",
    anchor.features = all_samples_features)
all_samples_list_sct <- IntegrateData(anchorset = cell_anchors, normalization.method = "SCT")

In [58]:
combined_sct <- RunPCA(all_samples_list_sct, verbose = FALSE, npcs = 50) # calculate the PCA for the integrated samples


In [ ]:

print(combined_sct[["pca"]], dims = 1:5, nfeatures = 5) # print the first 5 PCs


In [ ]:
DimPlot(combined_sct, reduction = "pca") # print the PCA plot


In [ ]:
ElbowPlot(combined_sct,ndims = 50) # print the elbow plot


## Clustering

In [ ]:
combined_sct <- RunUMAP(combined_sct, reduction = "pca", dims = 1:30, verbose = FALSE) # calculate the UMAP for the integrated samples
combined_sct <- FindNeighbors(combined_sct, reduction = "pca", dims = 1:30) # find the neighbors for the integrated samples
combined_sct <- FindClusters(combined_sct, resolution = c(0.1,0.15,0.2, 0.4, 0.6, 0.8, 1, 1.2), verbose = TRUE) # find the clusters for the integrated samples

## UMAP with no annotation and showing normalizaton across datasets

In [ ]:
DimPlot(combined_sct, reduction = "umap", group.by = "orig.ident", label = TRUE, repel = TRUE) # print the UMAP plot (also checks for the batch correction) 


In [ ]:
# check all resolutions for the integrated samples
DimPlot(combined_sct, reduction = "umap", group.by = "integrated_snn_res.0.15", label = TRUE, repel = TRUE)
DimPlot(combined_sct, reduction = "umap", group.by = "integrated_snn_res.0.1", label = TRUE, repel = TRUE)
DimPlot(combined_sct, reduction = "umap", group.by = "integrated_snn_res.0.2", label = TRUE, repel = TRUE)
DimPlot(combined_sct, reduction = "umap", group.by = "integrated_snn_res.0.4", label = TRUE, repel = TRUE)
DimPlot(combined_sct, reduction = "umap", group.by = "integrated_snn_res.0.6", label = TRUE, repel = TRUE)
DimPlot(combined_sct, reduction = "umap", group.by = "integrated_snn_res.0.8", label = TRUE, repel = TRUE)
DimPlot(combined_sct, reduction = "umap", group.by = "integrated_snn_res.1", label = TRUE, repel = TRUE)
DimPlot(combined_sct, reduction = "umap", group.by = "integrated_snn_res.1.2", label = TRUE, repel = TRUE)

In [85]:
Idents(combined_sct) <- combined_sct@meta.data$integrated_snn_res.0.4 # set the resolution to 0.4

In [ ]:
DimPlot(combined_sct, reduction = "umap", label = TRUE, repel = TRUE) # print the UMAP plot for the resolution 0.4

## Find Cluster Markers

In [ ]:
all_markers <- FindAllMarkers(combined_sct, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.2) # find the markers for all clusters

In [ ]:
# check the markers for all clusters
all_markers %>% group_by(cluster) %>% slice_max(n = 5, order_by = avg_log2FC) %>% ungroup() %>% arrange(cluster, avg_log2FC) %>% as.data.frame() 

# make a directory called results_seurat
dir.create("results_seurat")
# write the results to a csv file
all_markers %>% group_by(cluster) %>% slice_max(n = Inf, order_by = avg_log2FC) %>% ungroup() %>% arrange(cluster, desc(avg_log2FC)) %>% as.data.frame() %>% write.csv("results_seurat/all_markers_mt_removed.csv")

## Convert to Mouse Gene Symbols

In [ ]:
# Basic function to convert zebrafish to mouse gene names

zgGenes <- sib_1_seurat@assays$RNA %>% rownames() %>% unique() # get the zebrafish gene names
# This function uses biomaRt to convert zebrafish gene names to mouse gene names. 
# This is done by using the zebrafish and mouse ensembl mart datasets. 
# The function takes a vector of zebrafish gene names as input and returns 
# a data frame of zebrafish and mouse gene names.
convertDanioGeneList_Mouse <- function(x){ 
  require("biomaRt") # load biomaRt package
  mouse = useMart("ensembl", dataset = "mmusculus_gene_ensembl", host = "https://dec2021.archive.ensembl.org/") # use mouse mart
  danio = useMart("ensembl", dataset = "drerio_gene_ensembl",host = "https://dec2021.archive.ensembl.org/") # use zebrafish mart

  genesV2 = getLDS(attributes = c("ensembl_gene_id", "zfin_id_symbol"), 
                   filters = "zfin_id_symbol", # get zebrafish gene names
                   values = x , # use the zebrafish gene names
                   mart = danio,  # use the zebrafish mart
                   attributesL = c("mgi_symbol", "ensembl_gene_id", "description"), # get mouse gene names
                   martL = mouse, uniqueRows=T) # use the mouse mart
  
  colnames(genesV2)[colnames(genesV2)== "Gene.stable.ID"] <- "EnsmblID_Zebrafish" # rename columns
  colnames(genesV2)[colnames(genesV2)== "Gene.stable.ID.1"] <- "EnsmblID_Mouse" # rename columns
  
  # Check if the gene is not found
  if (length(genesV2) == 0) {
    print("No gene found for this input")
  } else { 
    return(genesV2) # return the genes
  }
}

# Run the function
Mouse_Genes <- convertDanioGeneList_Mouse(zgGenes)
# print the first 6 genes
head(Mouse_Genes)

In [ ]:
all_markers_mouse <- merge(all_markers, Mouse_Genes, by.x = "gene", by.y = "ZFIN.symbol", all.x = TRUE) # merge the markers with the mouse gene names
dim(all_markers_mouse)
dim(all_markers)
head(all_markers_mouse)

In [ ]:
# arrange all_markers_mouse by cluster and log2FC
all_markers_mouse <- all_markers_mouse %>% group_by(cluster) %>% slice_max(n = Inf, order_by = avg_log2FC) %>% ungroup() %>% arrange(cluster, desc(avg_log2FC)) %>% as.data.frame()
head(all_markers_mouse)
all_markers_mouse %>% write.csv("results_seurat/all_markers_mouse_mt_removed.csv")

## Assign cluster labels

In [ ]:
DimPlot(combined_sct, reduction = "umap", label = TRUE, repel = TRUE, label.size= 8) 

In [116]:
## markers from Junker's Fibroblast paper: https://www.nature.com/articles/s41588-022-01129-5
FB <- c("col1a2", "mdka", "col1a1a", "col1a1b", "col5a1", "dpt", "fn1b", "ccl25b", "sparc", "clu", "dcn", "mfap5", "postnb", "c4", "fn1a", "col12a1a", "c6.1", "si:ch1073-459j12.1", "htra1b", "pmp22a", "hapln1a", "ckba", "col6a3", "gstm.3", "col6a1", "tnfaip6", "tfa", "mmp2", "f3b", "rbp4", "col5a2a", "CR936442.1", "ifitm1", "sostdc1a", "anxa2a", "pcolcea", "olfml3b", "tmsb2", "si:ch211-105c13.3", "si:ch211-137i24.10", "c1qtnf5", "ppib", "si:ch211-106h4.12", "col6a2", "mgp", "cxcl12a", "edil3a", "dkk3b", "si:ch211-198c19.3", "fstl1b")
## markers for Valve Fibroblasts
VF <- c("zgc:153704", "abi3bpb", "krt4", "angptl7", "igfbp5b", "cyp26b1", "tnfaip6", "mgp", "rspo1", "fibinb", "aif1l", "krt91", "bhmt", "gyg1b", "si:dkey-4p15.3", "serpinh1b", "si:dkey-205h13.1", "tuft1a", "f13a1b", "ITGB1BP2", "crip1", "carhsp1", "uchl1", "fbln5", "krt15", "rbp5", "gch2", "pcolcea", "irs2b", "cbx7a", "snorc", "hspb8", "rpl22l1", "thbs1b", "ptx3a", "timp4.3", "csrp2", "laptm4b", "cebpd", "crip2", "mmp2", "dcn", "sec61g", "rps27.2", "tgm2l", "GSTZ1", "matn4", "hmgb3a", "MYO1D", "fxyd6l")
## markers for Endothelial cells (apnln)
EC_apln <- c("apnln", "hbegfb", "admb", "apln", "si:ch211-195b11.3", "sele", "itga2b", "thbs1b", "fgl2a", "FO681357.1", "pim2", "gp1bb", "cldn5b", "gcnt4a", "il11b", "si:ch211-105j21.9", "tln1", "si:ch211-214p13.9", "f5", "si:ch211-214p16.2", "si:ch211-153b23.5", "si:ch211-214p16.1", "rap1b", "blf", "Sep-12", "myh9a", "fam212ab", "plek", "lpp", "srgn", "pfn1", "hsp70l", "hacd3", "si:ch211-250g4.3", "fermt3b", "blvrb", "atp2b1a", "cbx7a", "hsp70.1", "hsp70.2", "crema", "zgc:171686", "mpl", "tpte", "lmo2", "arpc1b", "grasp", "hsp70.3", "myh11a", "si:ch211-222l21.1", "bmp16")
## markers for Endothelial cells (lyve1)
EC_lyve1 <- c("cxcl12a", "CU929150.1", "thy1", "lyve1a", "selenop", "cdh6", "si:dkey-203a12.9", "lyve1b", "id1", "si:ch211-105c13.3", "arl4aa", "si:ch211-152c2.3", "hapln3", "stab1", "loxl1", "ramp2", "etv2", "hsd3b7", "ctsla", "rgcc", "akap12a", "tppp3", "tcima", "ch25h", "fabp11a", "CR762483.1", "plvapb", "sox7", "gstt1b", "pde4ba", "lgmn", "plk2b", "hyal2b", "mrc1b", "rab11bb", "gpm6ab", "stab2", "dab2", "myct1a", "zfp36l1b", "ponzr1", "flt4", "clec14a", "ebf3a.1", "s1pr1", "carhsp1", "fxyd6l", "cldn11a", "ms4a17a.8", "igf2a")
## markers for Endothelial cells (plvapb)
EC_plvapb <- c("wu:fj16a03", "cxcl12b", "rbp2a", "plvapb", "cldn5b", "rgcc", "fabp11a", "id1", "tmsb1", "ldb2a", "tcima", "cavin1b", "rgs5b", "hopx", "cdh5", "kdrl", "scarb2a", "gig2j", "bcl6b", "bsg", "podxl", "mcamb", "pdgfbb", "jam2a", "ecscr", "si:dkey-126g1.9", "krt18", "ptprb", "epas1b", "si:ch211-156j16.1", "si:dkey-262k9.2", "CABZ01030107.1", "plvapa", "sox7", "cav1", "ch25h", "plpp3", "col4a1", "CABZ01075125.1", "gpm6ab", "fgd5a", "pecam1", "ca16b", "gpm6aa", "cavin2b", "TCIM", "limch1b", "igfbp1a", "clec14a", "oaz2a")
## markers for Cardiomyocytes (dediff.)
CM_dediff <- c("nppa", "nppb", "hsp90aa1.1", "ttn.2", "ttn.1", "myh6", "atp2a2a", "xirp1", "si:ch211-131k2.3", "hspb11", "her4.1", "mybpc3", "lmo7a", "si:ch211-270g19.5", "unc45b", "tcap", "ryr2b", "tnnc1b", "synpo2lb", "ldb3a", "pla1a", "sorbs1", "bves", "flnca", "ACTC1", "slc8a1a", "ndrg4", "trdn", "myh7l", "pdk2a", "pabpc4", "myom1b", "cox7c", "hspb8", "cited4b", "ATP5MD", "desma", "cox6a2", "zgc:101840", "mt-co1", "bag3", "ITGB1BP2", "kcnh6a", "tnni1b", "tnnt2a", "laptm4b", "smtnl1", "aldoab", "pygmb", "cox7a1")
## markers for Cardiomyocytes (Atrium)
CM_atrium <- c("tnnc1b", "myh6", "si:ch211-270g19.5", "tcap", "tnni1b", "aldoab", "tnnt2a", "smtnl1", "cmlc1", "gapdh", "tnnc1a", "nppa", "ddx54", "ckma", "zgc:101840", "mybphb", "myl7", "tpm4a", "actc1a", "acta1b", "idh2", "cox5b2", "mpc1", "mdh2", "uqcrfs1", "mdh1aa", "ldhba", "ak1", "ptp4a3", "nmrk2", "atp5mc3b", "atp5mc1", "calm3a", "atp5pd", "atp5meb", "atp5f1b", "chchd10", "adprhl1", "fhl2a", "ckbb", "atp5mc3a", "lmod2b", "si:dkey-51e6.1", "cox5aa", "ndufb10", "mybpc3", "zgc:85722", "slc25a3b", "atp5if1b", "hbaa1")
## markers for Cardiomyocytes (Ventricle)
CM_ventricle <- c("tnni4a", "CR926459.1", "fabp3", "myl7", "ak1", "cmlc1", "actc1a", "acta1b", "myh7l", "ndufa4", "gapdh", "ckma", "eno3", "tnnt2a", "atp5pd", "cox5b2", "cox7a1", "aldoaa", "tnnc1a", "tnni1b", "tpm4a", "cox4i1l", "mdh2", "ppdpfa", "mustn1b", "atp5mc1", "idh2", "nme2b.2", "rbpms2b", "pgam2", "ldb3b", "hspb11", "cox6b1", "atp5meb", "cox6a2", "ATP5MD", "mpc1", "gamt", "atp5po", "atp5mc3b", "zgc:193541", "tpi1b", "uqcr10", "slc25a5", "csrp3", "atp5if1b", "atp5f1d", "atp5f1e", "cox7b", "mdh1aa")
## markers for Endocardium (Atrium)
encar_endo_atrium <- c("zgc:158343", "spock3", "im:7152348", "ptgs2a", "ptgs2b", "id2b", "vcam1b", "aqp8a.1", "mycb", "her6", "krt18", "grb10a", "clic2", "si:ch211-248e11.2", "gpr182", "klf2a", "cxcl18b", "fosl1a", "tmem88b", "dusp5", "glulb", "si:ch73-335l21.4", "akap12b", "ctsla", "fosb", "ackr4b", "igf2b", "bzw1b", "si:ch211-153b23.5", "tmem88a", "zfp36l1b", "ramp2", "errfi1a", "ponzr1", "fosab", "si:ch211-145b13.6", "btg2", "junba", "cyp2ad2", "nppc", "lpl", "slc22a31", "sat1a.2", "jun", "junbb", "tmem2", "vwf", "kctd12.2", "spint2", "pim1")
## markers for Endocardium (frzb)
encar_frzb <- c("vwf", "si:ch211-153b23.5", "c2cd4a", "edn2", "cfd", "frzb", "ecscr", "glulb", "efemp2b", "calm1a", "cyr61", "nr4a2b", "cpamd8", "nr4a1", "cdh5", "s100v2", "appa", "cyr61l2", "hbegfa", "si:ch211-248e11.2", "entpd8", "her9", "klf2a", "ptmaa", "ptgs2a", "krt15", "ehd2b", "crip2", "spock3", "lmo2", "junba", "s100a10b", "id2b", "clec14a", "icn", "clic2", "cav1", "gpd1b", "fosl2", "tie1", "gadd45ba", "errfi1a", "BX663503.3", "sox7", "net1", "ackr4b", "serpine1", "sult2st3", "CABZ01030107.1", "efnb2a")
## markers for Endocardium (Ventricle)
encar_endo_ventricle <- c("si:ch73-86n18.1", "aqp8a.1", "spock3", "mb", "fabp11a", "epas1b", "ramp2", "si:ch211-145b13.6", "f8", "her6", "cdh5", "fosab", "si:ch211-248e11.2", "eppk1", "socs3a", "tmem88b", "podxl", "cyp1b1", "grb10a", "si:ch73-335l21.4", "lpl", "jun", "si:ch211-160j14.3", "spon1b", "tmem88a", "akap12b", "klf2b", "id2b", "zgc:64106", "egfl7", "ecscr", "tspan4b", "fam174b", "kdrl", "junba", "ackr4b", "zgc:152791", "si:dkey-261h17.1", "si:dkeyp-97a10.2", "junbb", "nppc", "slc22a31", "klf6a", "clic2", "appa", "flt1", "tfpia", "ier2b", "im:7152348", "ccdc187")
## markers for Epicardium (Atrium)
EPIC_atrium <- c("gstm.3", "s100a10a", "fn1b", "krt15", "mmp2", "si:ch211-105c13.3", "krt4", "frzb", "mmel1", "anxa2a", "ppdpfa", "krt94", "rbp4", "si:ch211-286o17.1", "anxa1a", "CR318588.3", "dkk3b", "tcf21", "postnb", "slc29a1b", "tmsb1", "phactr4a", "podxl", "c6.1", "myh10", "tuba1a", "f3b", "timp2a", "tbx18", "endouc", "cxcl14", "ckba", "aldh1a2", "col6a1", "c1qtnf5", "hsd11b2", "c7a", "si:ch211-156j16.1", "id2a", "rn7sk", "acvrl1", "spaca4l", "mxra8b", "mfap5", "s100a10b", "c4", "lxn", "flnb", "olfml3b", "cav1")
## markers for Epicardium (Ventricle)
EPIC_ventricle <- c("si:ch211-106h4.12", "gstm.3", "krt15", "postnb", "s100a10a", "fn1b", "endouc", "mmp2", "tfa", "krt4", "krt94", "frzb", "si:ch211-105c13.3", "cldnc", "tmsb1", "si:ch211-198c19.3", "adh8a", "si:ch1073-459j12.1", "mdka", "c6.1", "anxa2a", "col1a2", "podxl", "si:ch211-137i24.10", "col1a1a", "spaca4l", "igfbp5b", "col1a1b", "rbp4", "eppk1", "sparc", "anxa1a", "dkk3b", "cygb1", "col6a3", "tgm2b", "si:dkey-8k3.2", "sostdc1a", "mgp", "aldh1a2", "c4", "crip1", "zgc:123068", "ppdpfa", "hsd11b2", "fn1a", "col5a1", "col6a1", "colec12", "si:ch211-286o17.1")
## markers for Macrophages
MACROPHAGES <- c("grn1", "grn2", "ccl35.1", "cd74a", "c1qb", "ctsd", "c1qc", "lygl1", "lgals3bpb", "si:ch211-147m6.2", "marco", "fcer1gl", "ctss2.2", "mfap4", "NPC2 (1 of many)", "si:ch211-147m6.1", "cd74b", "grna", "si:busm1-266f07.2", "ctss2.1", "atp6v0ca", "hmox1a", "zgc:174904", "si:dkey-5n18.1", "ctsba", "rasgef1ba", "lgmn", "c1qa", "psap", "sqstm1", "CR855311.1", "ctsz", "pfn1", "atp6v1g1", "si:ch211-212k18.7", "tspan36", "ctsc", "si:dkey-27i16.2", "mrc1b", "cd9b", "zgc:92066", "ccl34a.4", "tppp", "cotl1", "ncf1", "cfp", "laptm5", "coro1a", "si:ch211-194m7.4", "scpep1")
## markers for Monocytes
MONOCYTES <- c("ccl35.1", "epdl1", "si:ch211-214p16.1", "si:busm1-266f07.2", "si:ch211-214p16.2", "ccl35.2", "CU459094.3", "cd74a", "arpc1b", "cd74b", "tmsb1", "anxa3b", "sftpbb", "id2a", "mpeg1.1", "mhc2dab", "cxcr4b", "si:dkey-33i11.9", "zgc:92066", "pfn1", "hspbp1", "DNAJA4", "si:dkey-27i16.2", "stip1", "cxcl11.6", "aif1l", "hspa4a", "nr4a3", "syngr3b", "sqstm1", "zgc:152863", "rhbdd1", "hspe1", "BX004816.2", "pdcd4b", "zgc:103700", "chchd2", "actb2", "zdhhc18b", "psap", "rasgef1ba", "ctss2.2", "zgc:64051", "spi1b", "zgc:136870", "coro1a", "hspa8", "hsp90aa1.2", "wbp2", "ckbb")
## markers for Myelin cells
MYELIN_CELLS <- c("scn4ab", "cd59", "mbpa", "mpz", "mbpb", "plp1b", "BX936284.1", "anxa13l", "apoeb", "si:rp71-19m20.1", "sncga", "nrgna", "stmn1b", "itgb4", "scn1ba", "fstl3", "cldnk", "sox10", "CABZ01068367.1", "pdgfbb", "si:ch211-286c4.6", "nanos1", "entpd3", "timd4", "lim2.2", "foxd3", "acbd7", "pou3f1", "cldn19", "ugt8", "gldn", "si:ch211-197g15.7", "fabp2", "znf536", "ninj2", "depdc7", "si:ch211-137a8.4", "tlcd1", "elovl1a", "si:ch211-234p6.5", "col15a1b", "egr2b", "gpm6ab", "fstb", "nr4a2b", "dhrs3b", "phlda2", "calm1a", "cd9b", "vwa1")
## markers for Neuronal cells
NEURONAL_CELLS <- c("vipb", "elavl4", "stmn1b", "syt1a", "tuba1c", "gap43", "snap25a", "stmn2a", "sncga", "rtn1b", "mllt11", "crhbp", "slc5a7a", "anxa13l", "zgc:65894", "stmn2b", "ddc", "chata", "gng3", "FO907089.1", "phox2a", "pcsk1nl", "scg2b", "atp1b2a", "cpe", "nsg2", "vip", "vgf", "hpcal4", "atp1a3a", "cplx2", "phox2bb", "prph", "synpr", "map1b", "maptb", "rab6bb", "tuba2", "syt11a", "chrna3", "stxbp1a", "si:ch73-119p20.1", "ret", "vamp2", "cplx2l", "cntnap2a", "sypa", "eef1a1a", "stx1b", "elavl3")
## markers for Neutrophils
NEUTROPHILS <- c("lyz", "lect2l", "BX908782.2", "npsn", "si:ch211-117m20.5", "mmp13a.1", "si:ch211-9d9.1", "scpp8", "si:ch1073-67j19.1", "mmp9", "pnp5a", "mpx", "ncf1", "fcer1gl", "pfn1", "scinlb", "cfl1l", "cotl1", "plekhf1", "cpa5", "illr4", "myh9a", "adam8a", "si:dkey-102g19.3", "lta4h", "timp2b", "il6r", "si:ch73-54b5.2", "cfbl", "vsir", "alox5ap", "nccrp1", "arg2", "si:ch211-284o19.8", "CU499336.2", "si:ch1073-443f11.2", "s1pr4", "arpc1b", "DNAJA4", "antxr2a", "si:ch211-136m16.8", "tmsb1", "si:ch73-343l4.8", "srgn", "zgc:77112", "coro1a", "stip1", "cd7al", "si:ch73-248e21.5", "CT030188.1")
## markers for Perivascular cells
PERIVASCULAR_CELLS <- c("TCIM", "cxcl12b", "pdgfrb", "rasl12", "BX901920.1", "kcne4", "rgs5b", "rgs4", "agtr2", "TPM1", "si:ch211-198c19.3", "ifitm1", "fxyd6l", "slc20a1a", "calm2b", "notch3", "cd248a", "tagln", "si:ch73-193i22.1", "col18a1b", "mcamb", "atp1a1b", "stk17al", "rgs5a", "mustn1a", "myh11a", "pcdh18a", "plp1b", "fhl3b", "sh3d21", "ccdc3b", "prx", "snai1a", "sparc", "si:ch211-251b21.1", "si:ch211-270g19.5", "slc7a2", "myo1b", "jam3a", "arhgap36", "AL954322.2", "BX908750.1", "atp1b4", "en1b", "CABZ01068367.1", "oscp1a.1", "ppp1r14aa", "rftn2", "rassf4", "spdya")
## markers for Proliferating cells
PROLIFERATING_CELLS <- c("pcna", "stmn1a", "hmgb2b", "DUT", "sumo3b", "rrm2.1", "si:ch211-288g17.3", "rpa3", "banf1", "rpa2", "lig1", "fen1", "sfrp5", "dnajc9", "rbbp4", "tyms", "rrm1", "top2a", "nutf2l", "cks1b", "hells", "chaf1a", "selenoh", "mki67", "zgc:110540", "mcm5", "mcm6", "ubr7", "si:ch211-156b7.4", "smc2", "si:ch211-66k16.27", "mcm4", "mcm3", "rrm2", "cdk1", "dek", "rpa1", "h2afx", "mcm2", "nasp", "cks2", "gins2", "asf1ba", "dhfr", "prim2", "rfc5", "rnaseh2a", "chtf8", "si:dkey-6i22.5", "dctd")
## markers for Smooth muscle cells
SMOOTH_MUSCLE_CELLS <- c("rgs5a", "acta2", "TPM1", "C11orf96", "tagln", "myh11a", "krt91", "itih1", "myl6", "myl9a", "BX663503.3", "vim", "si:dkey-164f24.2", "ctgfa", "lmod1b", "flna", "cfd", "tpm2", "si:busm1-57f23.1", "zgc:77517", "mdkb", "tns1b", "si:ch211-1a19.3", "mylka", "gch2", "slmapb", "cyr61", "lpp", "dnajb4", "CR855337.1", "fhl1a", "fhl3b", "fbln5", "alcama", "tmsb4x", "c2cd4a", "htra1a", "phlda2", "anxa1a", "cav2", "cst3", "zgc:92162", "f3a", "sort1a", "cald1b", "kng1", "epas1a", "wfdc2", "tpm4b", "fhl2a")
## markers for T-cells
T_CELLS <- c("si:ch211-214p16.1", "ccl34b.4", "ccl36.1", "ccl38.6", "DNAJA4", "ccr9a", "CR936442.1", "cxcr4b", "hspbp1", "zgc:64051", "rgs13", "pfn1", "nkl.1", "pdcd4b", "hspa4a", "sla2", "cebpb", "cyb5a", "coro1a", "stip1", "b2m", "si:dkey-27i16.2", "si:dkey-109a10.2", "si:ch211-202a12.4", "chchd2", "hsp90aa1.2", "rhbdd1", "cxcr4a", "tnfb", "calm1b", "tnfrsf9b", "ptprc", "si:ch211-105j21.9", "dusp2", "arpc1b", "sh2d1ab", "FP236356.1", "wasb", "BX120005.1", "capgb", "si:dkey-262k9.2", "si:ch211-165d12.4", "BX005105.1", "laptm5", "oser1", "mcl1a", "wbp2", "fkbp4", "scinlb", "si:dkey-177p2.6")

In [ ]:
# render the plot to be bigger
options(repr.plot.width=18, repr.plot.height=32)
# make a heatmap usingd DoHeatmap using the markers mentioned above
DoHeatmap(object = combined_sct, features = c(FB, VF, EC_apln, EC_lyve1, EC_plvapb, CM_dediff, CM_atrium, CM_ventricle, encar_endo_atrium, encar_frzb, encar_endo_ventricle, EPIC_atrium, EPIC_ventricle, MACROPHAGES, MONOCYTES, MYELIN_CELLS, NEURONAL_CELLS, NEUTROPHILS, PERIVASCULAR_CELLS, PROLIFERATING_CELLS, SMOOTH_MUSCLE_CELLS, T_CELLS), group.by = "integrated_snn_res.0.4")
options(repr.plot.width=12, repr.plot.height=12)


In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
# Valve genes accodring to: https://www.nature.com/articles/s42003-021-02571-7--  by myra

valve_markers_nat <- unique(c("fgfr2","tgfb2", "fn1b", "wt1a", "fgfr4", "smad6b", "fgfr3", "gata5", "bmpr2b", "piezo2a.2"))
# Epicardial markers according to : https://www.sciencedirect.com/science/article/pii/S1534580720300551
epicardial_markers <- unique(c("fstl1a", "fstl1b", "cav1", "aldh1a2", "tcf21","tgm2b", "sema3fb","tbx18", "adma", "jam2b", "wt1b", "cldn11a", "elnb", "mylka"))
VlnPlot(object = combined_sct, features = valve_markers_nat, stack = TRUE, group.by = "integrated_snn_res.0.4") + NoLegend()
VlnPlot(object = combined_sct, features = epicardial_markers, stack = TRUE, group.by = "integrated_snn_res.0.4") + NoLegend()


In [ ]:
genes_to_annotate <- c("myh6", "cdh5", "pecam1", "pdgfra", "mslnb", "myh11a", "kcnj8", "adgre1", "itgal", "naaa", "s100a9", "cd3g", "ms4a1", "plp1", "hbb-bt", "hba-a1", "postnb") # genes to annotate to check for clusters
VlnPlot(object = combined_sct, features = genes_to_annotate, stack = TRUE, group.by = "integrated_snn_res.0.4") + NoLegend()

In [ ]:
# check the number of cells 
table(combined_sct@meta.data$integrated_snn_res.0.4, combined_sct@meta.data$orig.ident)

In [ ]:
# cluster	name
# 0	CM_1
# 1	Neurons
# 2	CM_2
# 3	EC_Endocardium
# 4	Erythrocytes
# 5	CM_3
# 6	FB_1
# 7	FB_2
# 8	epicardium
# 9	Immune cells
# 10 Neurons?
# 11 Skeletal muscle
# 12 unknown_2
# 13 epithelial cells



In [127]:
# allocate cell types to the clusters
combined_sct@meta.data$cell_type <- NULL
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 0] <- "CM_1"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 1] <- "Neurons"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 2] <- "CM_2"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 3] <- "EC_Endocardium"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 4] <- "Erythrocytes"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 5] <- "CM_3"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 6] <- "FB_1"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 7] <- "FB_2"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 8] <- "epicardium"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 9] <- "Immune cells"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 10] <- "Neurons?"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 11] <- "Skeletal muscle"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 12] <- "unknown_2"
combined_sct@meta.data$cell_type[combined_sct@meta.data$integrated_snn_res.0.4 == 13] <- "epithelial cells"





In [129]:
Idents(combined_sct) <- combined_sct@meta.data$cell_type

In [ ]:
# change the plot size
options(repr.plot.width=12, repr.plot.height=12)
DimPlot(combined_sct, reduction = "umap", label = TRUE, label.size = 6, repel = TRUE, pt.size = 2)

# check gene markers

In [ ]:
# read marker genes from Burkhard et al. 2018-- https://elifesciences.org/articles/31515
# read sheet Ventricle 
Burkhard_ventricle <- openxlsx::read.xlsx("Burkhard_et_al_fig4.xlsx", sheet="Ventricle")
# make first row as column names
colnames(Burkhard_ventricle) <- Burkhard_ventricle[1,]
# remove first row
Burkhard_ventricle <- Burkhard_ventricle[-1,]

# convert to numeric
Burkhard_ventricle$`PValue` <- as.numeric(Burkhard_ventricle$`PValue`)
Burkhard_ventricle$`logFC` <- as.numeric(Burkhard_ventricle$`logFC`)

Burkhard_ventricle %>% head

# count how many genes are significant 
sum(Burkhard_ventricle$`PValue` < 0.05)



In [ ]:
# read marker genes from Burkhard et al. 2018 
# read sheet Atrium
Burkhard_atrium <- openxlsx::read.xlsx("Burkhard_et_al_fig4.xlsx", sheet="Atrium")
# make first row as column names
colnames(Burkhard_atrium) <- Burkhard_atrium[1,]
# remove first row
Burkhard_atrium <- Burkhard_atrium[-1,]

# convert to numeric
Burkhard_atrium$`PValue` <- as.numeric(Burkhard_atrium$`PValue`)
Burkhard_atrium$`logFC` <- as.numeric(Burkhard_atrium$`logFC`)

Burkhard_atrium %>% head

# count how many genes are significant 
sum(Burkhard_atrium$`PValue` < 0.05)


In [ ]:
# read marker genes from Burkhard et al. 2018 
# read sheet AV canal
Burkhard_AV_canal <- openxlsx::read.xlsx("Burkhard_et_al_fig4.xlsx", sheet="AV canal")
# make first row as column names
colnames(Burkhard_AV_canal) <- Burkhard_AV_canal[1,]
# remove first row
Burkhard_AV_canal <- Burkhard_AV_canal[-1,]

# convert to numeric
Burkhard_AV_canal$`PValue` <- as.numeric(Burkhard_AV_canal$`PValue`)
Burkhard_AV_canal$`logFC` <- as.numeric(Burkhard_AV_canal$`logFC`)

Burkhard_AV_canal %>% head

# count how many genes are significant 
sum(Burkhard_AV_canal$`PValue` < 0.05)

In [ ]:
# read marker genes from Burkhard et al. 2018 
# read sheet SA region
Burkhard_SA_region <- openxlsx::read.xlsx("Burkhard_et_al_fig4.xlsx", sheet="SA region")
# make first row as column names
colnames(Burkhard_SA_region) <- Burkhard_SA_region[1,]
# remove first row
Burkhard_SA_region <- Burkhard_SA_region[-1,]

# convert to numeric
Burkhard_SA_region$`PValue` <- as.numeric(Burkhard_SA_region$`PValue`)
Burkhard_SA_region$`logFC` <- as.numeric(Burkhard_SA_region$`logFC`)

Burkhard_SA_region %>% head

# count how many genes are significant 
sum(Burkhard_SA_region$`PValue` < 0.05)




In [ ]:
#get genes which are significant in all 4 regions and convert them to vector
Burkhard_ventricle %>% 
  filter(`PValue` < 0.05) %>% 
  dplyr::pull(`Gene ID`) %>% 
  as.character() -> ventricle_genes
Burkhard_atrium %>% filter(`PValue` < 0.05) %>% dplyr::pull(`Gene ID`)  %>% as.character() -> atrium_genes
Burkhard_AV_canal %>% filter(`PValue` < 0.05) %>% dplyr::pull(`Gene ID`)  %>% as.character() -> AV_canal_genes
Burkhard_SA_region %>% filter(`PValue` < 0.05) %>% dplyr::pull(`Gene ID`)  %>% as.character() -> SA_region_genes
ventricle_genes %>% head

In [ ]:
# render the plot to be bigger
options(repr.plot.width=18, repr.plot.height=32)
# make a heatmap usingd DoHeatmap using the markers mentioned above
DoHeatmap(object = combined_sct, features = c(ventricle_genes, atrium_genes,AV_canal_genes, SA_region_genes ), group.by = "integrated_snn_res.0.4")
DoHeatmap(object = combined_sct, features = c(ventricle_genes, atrium_genes,AV_canal_genes, SA_region_genes ))
options(repr.plot.width=12, repr.plot.height=12)

In [ ]:
# CM_1 Ventricle
# CM_2 Atrium


In [ ]:
DotPlot(object = combined_sct, features = c("has2", "bmp4", "tbx2b", "nppa", "vmhc", "myh6", "myh7", "tbx5", "vcana"))
DoHeatmap(object = combined_sct, features = c("has2", "bmp4", "tbx2b", "nppa", "vmhc", "myh6", "myh7", "tbx5", "vcana"))
VlnPlot(object = combined_sct, features = c("has2", "bmp4", "tbx2b", "nppa", "vmhc", "myh6", "myh7", "tbx5", "vcana"), stack=TRUE)
VlnPlot(object = combined_sct, features = c("has2", "bmp4", "tbx2b", "nppa", "vmhc", "myh6", "myh7", "tbx5", "vcana"), stack=TRUE, assay = "RNA")

In [ ]:
DotPlot(object = combined_sct, features = c("has2", "bmp4", "tbx2b", "nppa", "vmhc", "myh6", "myh7", "elnb", "isl1", "fgf10", "ltbp3", "mef2c", "tbx1", "eln2","vgo"))
VlnPlot(object = combined_sct, features = c("has2", "bmp4", "tbx2b", "nppa", "vmhc", "myh6", "myh7","elnb", "isl1","fgf10","ltbp3","mef2c", "tbx1","eln2", "vgo"), stack=TRUE)

# Proliferative cells

In [ ]:
s.genes <- cc.genes$s.genes
s.genes
g2m.genes <- cc.genes$g2m.genes
g2m.genes

In [ ]:

# Basic function to convert human to zebrafish gene names
convertHumanGeneList <- function(x){
require("biomaRt")
human = useMart("ensembl", dataset = "hsapiens_gene_ensembl", host = "https://dec2021.archive.ensembl.org") # use the archive version of ensembl
zebrafish = useMart("ensembl", dataset = "drerio_gene_ensembl", host = "https://dec2021.archive.ensembl.org") # use the archive version of ensembl
genesV2 = getLDS(attributes = c("hgnc_symbol"), # use the hgnc_symbol as the attribute to search for
                filters = "hgnc_symbol", # use the hgnc_symbol as the filter
                values = x , # x is the input vector of human genes
                mart = human, # human is the biomart object created above
                attributesL = c("zfin_id_symbol"), # zfin_id_symbol is the zebrafish gene symbol
                martL = zebrafish, # zebrafish is the biomart object created above
                uniqueRows=T) # only return unique rows

# Print the first 6 genes found to the screen
print(head(genesV2))
return(genesV2)
}


In [ ]:
s.genes_mouse <- convertHumanGeneList(s.genes)
g2m_mouse  <- convertHumanGeneList(g2m.genes)


In [ ]:
# calculate cell cycle scores using the mouse genes
combined_sct <- CellCycleScoring(combined_sct, s.features = s.genes_mouse$ZFIN.symbol, g2m.features = g2m_mouse$ZFIN.symbol, set.ident = FALSE,assay= "RNA")

In [ ]:
# calculate the percentage of _cycle in each cell type according to condition
cell_type_percentages_cell_cycle <- prop.table(table(combined_sct@meta.data$cell_type, combined_sct@meta.data$Phase, combined_sct@meta.data$condition))*100
cell_type_percentages_cell_cycle <- as.data.frame(cell_type_percentages_cell_cycle)
cell_type_percentages_cell_cycle %>% head
cell_type_percentages_cell_cycle <- cell_type_percentages_cell_cycle %>% dplyr::rename(cell_type = Var1, cell_cycle = Var2, condition = Var3)
cell_type_percentages_cell_cycle %>% head
# plot these percentages using ggplot2
ggplot(cell_type_percentages_cell_cycle, aes(x = cell_type, y = Freq, fill = cell_cycle)) +
  geom_bar(stat = "identity", position = "fill") + 
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.5)) +
  labs(x = "Condition", y = "Percentage of cells", fill = "Cell type")+facet_grid(~condition)

In [ ]:
ggplot(cell_type_percentages_cell_cycle, aes(x = cell_type, y = Freq, fill = cell_cycle)) +
  geom_bar(stat = "identity", position = "dodge") + 
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.5)) +
  labs(x = "Condition", y = "Percentage of cells", fill = "Cell type")+facet_grid(~condition)

In [33]:
write.csv(cell_type_percentages_cell_cycle, file = "results_seurat/cell_type_percentages_cell_cycle.csv") # save the percentages to a csv file

# Sub cluster cell types

In [ ]:
CM_cells  <- subset(combined_sct, subset = cell_type == "CM_1" | cell_type == "CM_2"| cell_type == "CM_3")
CM_cells

In [35]:
CM_cells@meta.data %>% head

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.mt_after_removal,nCount_SCT,nFeature_SCT,scDblFinder.score,scDblFinder.class,integrated_snn_res.0.4,⋯,seurat_clusters,integrated_snn_res.0.2,integrated_snn_res.0.1,integrated_snn_res.0.15,cell_type,condition,replicate,S.Score,G2M.Score,Phase
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<chr>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
AAACAGCCAGCTAACC-1_1,sib_1,491,407,0.4056795,0,484,400,0.03564680,singlet,2,⋯,3,2,2,2,CM_2,sib,1,-0.010554090,-0.012403101,G1
AAACAGCCATTAAGCT-1_1,sib_1,595,501,1.3266998,0,582,492,0.32570428,singlet,5,⋯,9,6,4,6,CM_3,sib,1,0.013786280,-0.010335917,S
AAACGGATCCCTCGCA-1_1,sib_1,516,338,0.0000000,0,510,332,0.10627147,singlet,0,⋯,2,1,1,1,CM_1,sib,1,-0.003298153,0.008755130,G2M
AAAGCAAGTGAAACAA-1_1,sib_1,541,392,1.9927536,0,538,390,0.13271956,singlet,5,⋯,9,6,4,6,CM_3,sib,1,-0.011213720,0.048487612,G2M
AAAGCACCACTAAGTT-1_1,sib_1,467,358,0.6382979,0,472,357,0.02887778,singlet,5,⋯,12,6,4,6,CM_3,sib,1,-0.006596306,0.022678219,G2M
AAAGGACGTCGAAGTC-1_1,sib_1,406,312,0.4901961,0,422,310,0.06708326,singlet,0,⋯,8,1,1,1,CM_1,sib,1,0.019722955,-0.007235142,S


In [ ]:
CM_cells <- SCTransform(CM_cells, verbose = FALSE)
CM_cells <- RunPCA(CM_cells, verbose = FALSE, npcs = 50)
ElbowPlot(CM_cells,ndims = 50)

In [ ]:
CM_cells <- RunUMAP(CM_cells, dims = 1:50)


In [ ]:
CM_cells <- FindNeighbors(object = CM_cells)
CM_cells <- FindClusters(CM_cells, resolution = c(0.2, 0.4, 0.6, 0.8, 1.0, 1.2))

In [ ]:
DimPlot(CM_cells, reduction = "umap", group.by = "orig.ident", label = TRUE, repel = TRUE)



In [ ]:
# CM_cells@meta.data %>% head

In [ ]:
DimPlot(CM_cells, reduction = "umap", group.by = "SCT_snn_res.0.2", label = TRUE, repel = TRUE)
DimPlot(CM_cells, reduction = "umap", group.by = "SCT_snn_res.0.4", label = TRUE, repel = TRUE)
DimPlot(CM_cells, reduction = "umap", group.by = "SCT_snn_res.0.6", label = TRUE, repel = TRUE)
DimPlot(CM_cells, reduction = "umap", group.by = "SCT_snn_res.0.8", label = TRUE, repel = TRUE)
DimPlot(CM_cells, reduction = "umap", group.by = "SCT_snn_res.1", label = TRUE, repel = TRUE)
DimPlot(CM_cells, reduction = "umap", group.by = "SCT_snn_res.1.2", label = TRUE, repel = TRUE)

In [19]:
AV_canal_genes_GO <-c("anxa5b", "apcdd1l", "aplnra", "aplnrb", "axin2", "bmp4", "cldn5a", "cldn5b", "cthrc1a", "cthrc1b", "dmgdh", "doc2b", "dpt", "dusp2", "egfl7", "elnb", "enpp2", "esama", "esamb", "fbln2", "gata5", "has2", "hspg2", "ipo11", "itga4", "itga5", "jag1a", "jag1b", "junba", "klf2a", "klf2b", "lef1", "lmna", "loxl2a", "loxl2b", "ncam1a", "ncam1b", "nfatc1", "nrg1", "nrg2a", "nrg2b", "nkd1", "notch1a", "notch1b", "notum1a", "notum1b", "ociad2", "pdgfaa", "pdlim3a", "pdlim3b", "piezo1", "plek", "prc1a", "prc1b", "prrx1b", "rnd2", "serpine2", "sgk1", "snai1a", "snai1b", "sost", "spp1", "tbx20", "tbx2b", "tbx3a", "tbx3b", "tgfb2", "tgfbi", "tgm2a", "tgm2b", "twist1a", "twist1b", "upp1", "vcana", "vcanb", "wnt16", "wnt4", "wnt9b", "zbtb20")

In [ ]:
DoHeatmap(object = CM_3_cells, features = c(AV_canal_genes_GO))

## sub cluster EC_Endocardium

In [ ]:
EC_Endo_cells <- subset(combined_sct, subset = cell_type == "EC_Endocardium" )
EC_Endo_cells

In [ ]:
EC_Endo_cells <- SCTransform(EC_Endo_cells, verbose = FALSE)
EC_Endo_cells <- RunPCA(EC_Endo_cells, verbose = FALSE, npcs = 50)
ElbowPlot(EC_Endo_cells,ndims = 50)

In [ ]:
EC_Endo_cells <- RunUMAP(EC_Endo_cells, dims = 1:50)

In [ ]:
EC_Endo_cells <- FindNeighbors(object = EC_Endo_cells)
EC_Endo_cells <- FindClusters(EC_Endo_cells, resolution = c(0.2, 0.4, 0.6, 0.8, 1.0, 1.2))

In [ ]:
DimPlot(EC_Endo_cells, reduction = "umap", group.by = "orig.ident", label = TRUE, repel = TRUE)


In [ ]:
DimPlot(EC_Endo_cells, reduction = "umap", group.by = "SCT_snn_res.0.2", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(EC_Endo_cells, reduction = "umap", group.by = "SCT_snn_res.0.4", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(EC_Endo_cells, reduction = "umap", group.by = "SCT_snn_res.0.6", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(EC_Endo_cells, reduction = "umap", group.by = "SCT_snn_res.0.8", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(EC_Endo_cells, reduction = "umap", group.by = "SCT_snn_res.1", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(EC_Endo_cells, reduction = "umap", group.by = "SCT_snn_res.1.2", label = TRUE, repel = TRUE, label.size = 6)

In [ ]:
Idents(EC_Endo_cells) <- "SCT_snn_res.0.6"
DimPlot(EC_Endo_cells, reduction = "umap", group.by = "SCT_snn_res.0.6", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(EC_Endo_cells, reduction = "umap", label = TRUE, repel = TRUE, label.size = 6)

In [ ]:
EC_Endo_cells_markers <- FindAllMarkers(EC_Endo_cells, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)
EC_Endo_cells_markers %>% head
EC_Endo_cells_markers %>% group_by(cluster) %>% top_n(n = 5, wt = avg_log2FC) %>% ungroup() %>% arrange(cluster, avg_log2FC) %>% head(20)
EC_Endo_cells_markers %>% group_by(cluster) %>% arrange(cluster, desc(avg_log2FC)) %>% write.csv(file = "results_seurat/EC_Endo_cells_markers.csv")

In [ ]:
FeaturePlot(EC_Endo_cells, features = c("notch1b"), label=TRUE, reduction = "umap", label.size = 6)
VlnPlot(EC_Endo_cells, features = c("notch1b"), split.by = "condition")

In [ ]:
# render the plot to be bigger
# options(repr.plot.width=18, repr.plot.height=32)
# make a heatmap usingd DoHeatmap using the markers mentioned above
DoHeatmap(object = EC_Endo_cells, features = c(FB, VF, EC_apln, EC_lyve1, EC_plvapb, CM_dediff, CM_atrium, CM_ventricle, encar_endo_atrium, encar_frzb, encar_endo_ventricle))
# options(repr.plot.width=12, repr.plot.height=12)
DoHeatmap(object = EC_Endo_cells, features = c(VF, valve_markers_nat))
options(repr.plot.width=12, repr.plot.height=12)

In [ ]:
DoHeatmap(object = EC_Endo_cells, features = c(VF, valve_markers_nat, "spns1", "spns2"), group.by = "condition")

In [ ]:
DotPlot(object = EC_Endo_cells, features = unique(c(VF, valve_markers_nat, "spns1", "spns2")), group.by = "SCT_snn_res.0.6", split.by = "condition")+ RotatedAxis()
# VlnPlot(object = combined_sct, features = genes_to_annotate, stack = TRUE, group.by = "integrated_snn_res.0.4") + NoLegend()

In [ ]:
EC_Endo_cells@meta.data$EC_identity <- NULL
EC_Endo_cells@meta.data$EC_identity[EC_Endo_cells@meta.data$`SCT_snn_res.0.6`==0] <- "EC_1"
EC_Endo_cells@meta.data$EC_identity[EC_Endo_cells@meta.data$`SCT_snn_res.0.6`==1] <- "EC_2"
EC_Endo_cells@meta.data$EC_identity[EC_Endo_cells@meta.data$`SCT_snn_res.0.6`==2] <- "Valve_cells"
EC_Endo_cells@meta.data$EC_identity[EC_Endo_cells@meta.data$`SCT_snn_res.0.6`==3] <- "EC_3"
EC_Endo_cells@meta.data$EC_identity[EC_Endo_cells@meta.data$`SCT_snn_res.0.6`==4] <- "EC_4"

head(EC_Endo_cells@meta.data)

DimPlot(EC_Endo_cells, group.by = "EC_identity", label = TRUE, repel = TRUE, label.size = 6)

In [ ]:
table(EC_Endo_cells@meta.data$condition)
table(EC_Endo_cells@meta.data$condition, EC_Endo_cells@meta.data$`EC_identity`)

In [13]:
Idents(EC_Endo_cells) <- "EC_identity"

In [ ]:

DefaultAssay(EC_Endo_cells) = "RNA"
DEG_Libra_edger_lrt_broad_EC_endo <- run_de(input = EC_Endo_cells, meta=meta, replicate_col = "replicate", cell_type_col = "EC_identity",
       label_col = "condition",  n_threads = 8)
DefaultAssay(EC_Endo_cells)  <- "integrated"

In [ ]:
DEG_Libra_edger_lrt_broad_EC_endo <- DEG_Libra_edger_lrt_broad_EC_endo %>% group_by(cell_type) %>% arrange(desc(abs(avg_logFC)),.by_group = TRUE, p_val_adj)
DEG_Libra_edger_lrt_broad_EC_endo%>% head
write.csv(DEG_Libra_edger_lrt_broad_EC_endo, "results_seurat/mut_vs_sib_libra_EdgeR_Lrt_broad_mt_removed_EC_endo.csv")

## sub cluster FBs

In [ ]:
FB_cells <- subset(combined_sct, subset = cell_type == "FB_1" | cell_type == "FB_2" )
FB_cells
FB_cells <- SCTransform(FB_cells, verbose = FALSE)
FB_cells <- RunPCA(FB_cells, verbose = FALSE, npcs = 50)
ElbowPlot(FB_cells,ndims = 50)

In [ ]:
FB_cells <- RunUMAP(FB_cells, dims = 1:50)
FB_cells <- FindNeighbors(object = FB_cells)
FB_cells <- FindClusters(FB_cells, resolution = c(0.2, 0.4, 0.6, 0.8, 1.0, 1.2))

In [ ]:
DimPlot(FB_cells, reduction = "umap", group.by = "orig.ident", label = TRUE, repel = TRUE)


In [ ]:
DimPlot(FB_cells, reduction = "umap", group.by = "SCT_snn_res.0.2", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(FB_cells, reduction = "umap", group.by = "SCT_snn_res.0.4", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(FB_cells, reduction = "umap", group.by = "SCT_snn_res.0.6", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(FB_cells, reduction = "umap", group.by = "SCT_snn_res.0.8", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(FB_cells, reduction = "umap", group.by = "SCT_snn_res.1", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(FB_cells, reduction = "umap", group.by = "SCT_snn_res.1.2", label = TRUE, repel = TRUE, label.size = 6)

In [ ]:
Idents(FB_cells) <- "SCT_snn_res.0.4"
DimPlot(FB_cells, reduction = "umap", group.by = "SCT_snn_res.0.4", label = TRUE, repel = TRUE, label.size = 6)
DimPlot(FB_cells, reduction = "umap", label = TRUE, repel = TRUE, label.size = 6)

In [ ]:
FB_cells_markers <- FindAllMarkers(FB_cells, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)
FB_cells_markers %>% head
FB_cells_markers %>% group_by(cluster) %>% top_n(n = 5, wt = avg_log2FC) %>% ungroup() %>% arrange(cluster, avg_log2FC) %>% head(20)
FB_cells_markers %>% group_by(cluster) %>% arrange(cluster, desc(avg_log2FC)) %>% write.csv(file = "results_seurat/FB_cells_markers.csv")

In [ ]:
FeaturePlot(FB_cells, features = c("notch1b"), label=TRUE, reduction = "umap", label.size = 6)
VlnPlot(FB_cells, features = c("notch1b"), split.by = "condition")

In [ ]:
table(FB_cells@meta.data$condition)
table(FB_cells@meta.data$condition, FB_cells@meta.data$`SCT_snn_res.0.4`)

In [ ]:

DefaultAssay(FB_cells) = "RNA"
DEG_Libra_edger_lrt_broad_FB <- run_de(input = FB_cells, meta=meta, replicate_col = "replicate", cell_type_col = "SCT_snn_res.0.4",
       label_col = "condition",  n_threads = 8)
DefaultAssay(FB_cells)  <- "integrated"

## Transfer labels from different subclusters to the main clusters

In [ ]:
combined_sct@meta.data$cell_name <- rownames(combined_sct@meta.data)
combined_sct@meta.data %>% head()

In [ ]:
EC_Endo_cells@meta.data$cell_name  <- rownames(EC_Endo_cells@meta.data)
EC_Endo_cells %>% head()

In [ ]:
EC_Endo_cells@meta.data[,c("cell_name", "EC_identity")] %>% head

In [ ]:
DimPlot(EC_Endo_cells, label = TRUE)

In [ ]:
combined_sct@meta.data <- combined_sct@meta.data[,!colnames(combined_sct@meta.data) %in% c('EC_identity.x','EC_identity.y','EC_identity.x','EC_identity.y','cell_type_EC')]
combined_sct@meta.data %>% names()

In [ ]:
combined_sct@meta.data <- merge(combined_sct@meta.data, EC_Endo_cells@meta.data[,c("cell_name", "EC_identity")], by = "cell_name", all.x = TRUE) 
rownames(combined_sct@meta.data) <- combined_sct@meta.data$cell_name
combined_sct@meta.data %>% head

In [ ]:
# replace cell_type in combined_sct@meta.data dataframe with entried of EC identity wherever EC_identiy is not NA
combined_sct@meta.data$cell_type_EC <- combined_sct@meta.data$cell_type
combined_sct@meta.data$cell_type_EC[!is.na(combined_sct@meta.data$EC_identity)] <- combined_sct@meta.data$EC_identity[!is.na(combined_sct@meta.data$EC_identity)]
combined_sct@meta.data %>% head
combined_sct@meta.data$cell_type %>% unique()
combined_sct@meta.data[combined_sct@meta.data$cell_type=="EC_Endocardium",] %>% head(10)
combined_sct@meta.data$cell_type_EC %>% unique()

In [48]:
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "CM_1"] <- "Ventricle"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "CM_2"] <- "Atrium"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "CM_3"] <- "AV_canal"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "FB_2"] <- "SMC_BA"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Neurons"] <- "unknown_1"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Neurons?"] <- "Neurons"


In [68]:
combined_sct@meta.data$broad_class <- combined_sct@meta.data$cell_type_EC

In [70]:
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "Atrium"] <- "Myocardial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "AV_canal"] <- "Myocardial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "EC_1"] <- "Endocardial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "EC_2"] <- "Endocardial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "EC_3"] <- "Endocardial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "EC_4"] <- "Endocardial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "epicardium"] <- "Epithelial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "epithelial cells"] <- "Epithelial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "Erythrocytes"] <- "Erythrocytes"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "FB_1"] <- "Fibroblast_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "Immune cells"] <- "Immune_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "Neurons"] <- "Neuronal_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "Skeletal muscle"] <- "Muscle_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "SMC_BA"] <- "Muscle_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "unknown_1"] <- "Unknown"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "unknown_2"] <- "Unknown"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "Valve_cells"] <- "Endocardial_cells"
combined_sct@meta.data$broad_class[(combined_sct@meta.data$cell_type_EC)== "Ventricle"] <- "Myocardial_cells"

In [72]:
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Atrium"] <- "CM_Atrium"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "AV_canal"] <- "CM_AV_canal"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "EC_1"] <- "EnC_1"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "EC_2"] <- "EnC_2"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "EC_3"] <- "EnC_3"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "EC_4"] <- "EnC_4"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "epicardium"] <- "Ep_epicardium"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "epithelial cells"] <- "Ep_epithelial cells"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Erythrocytes"] <- "Erythrocytes"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "FB_1"] <- "FB"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Immune cells"] <- "Immune_cells"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Neurons"] <- "Neurons"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Skeletal muscle"] <- "Mus_Skeletal_muscle"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "SMC_BA"] <- "Mus_SMC_BA"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "unknown_1"] <- "unknown_1"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "unknown_2"] <- "unknown_2"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Valve_cells"] <- "EnC_Valve_cells"
combined_sct@meta.data$cell_type_EC[(combined_sct@meta.data$cell_type_EC)== "Ventricle"] <- "CM_Ventricle"


In [ ]:
DimPlot(combined_sct, reduction = "umap", group.by = "cell_type_EC", label = TRUE, repel = TRUE, label.size = 6)

In [74]:
# change order of combined_sct@meta.data to keep sib first then mutant
combined_sct@meta.data$condition <- factor(combined_sct@meta.data$condition,levels = c("sib", "mut"))
combined_sct@meta.data$orig.ident <- factor(combined_sct@meta.data$orig.ident,levels = c("sib_1","sib_2", "mut_1", "mut_2"))

In [ ]:
unique(combined_sct@meta.data$orig.ident)
unique(combined_sct@meta.data$condition)

In [76]:
combined_sct@meta.data<- combined_sct@meta.data[order(match(combined_sct@meta.data$cell_name,names(combined_sct@active.ident))),]

In [ ]:
identical(combined_sct@meta.data$cell_name,names(combined_sct@active.ident))

In [80]:
Idents(combined_sct) <- "cell_type_EC"

In [ ]:
DimPlot( combined_sct, label = TRUE, repel = TRUE, label.size = 6)

In [ ]:
DimPlot( combined_sct, group.by = "broad_class",label = TRUE, repel = TRUE, label.size = 6)

In [83]:
# save seurat object for shiny app
saveRDS(combined_sct, "results_seurat/combined_sct.rds")

# Markers for each cluster and subclusters after cell assignment

In [ ]:
# Find markers with all the labels
all_markers_cell_assigned <- FindAllMarkers(combined_sct, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)
all_markers_cell_assigned %>% head
# arrange by descending log2fc and cell name and write to csv file
all_markers_cell_assigned %>% arrange(cluster, desc(avg_log2FC)) %>% write.csv("results_seurat/cell_assigned/all_markers_cell_assigned.csv")

In [ ]:
all_markers_cell_assigned_mouse <- merge(all_markers_cell_assigned, Mouse_Genes[!duplicated(Mouse_Genes$ZFIN.symbol),], by.x = "gene", by.y = "ZFIN.symbol", all.x=TRUE)
head(all_markers_cell_assigned_mouse)

In [86]:
all_markers_cell_assigned_mouse %>% arrange(cluster, desc(abs(avg_log2FC))) %>% write.csv("results_seurat/cell_assigned/all_markers_cell_assigned_mouse.csv")

In [ ]:
# make plot bigger
options(repr.plot.width=24, repr.plot.height=36)

all_markers_cell_assigned %>%
    group_by(cluster) %>%
    top_n(n = 20, wt = avg_log2FC) -> top10
dev.copy(pdf, "results_seurat/cell_assigned/all_markers_cell_assigned.pdf", width = 24, height = 36)
DoHeatmap(combined_sct, features = top10$gene) + NoLegend()
dev.off()
# make plots small again
options(repr.plot.width=12, repr.plot.height=12)

# Differential Expression Analysis using Libra

In [ ]:
DefaultAssay(combined_sct) = "RNA"
DEG_Libra_edger_lrt_cell_assigned<- run_de(input = combined_sct, meta=meta, replicate_col = "replicate", cell_type_col = "cell_type_EC",
       label_col = "condition",  n_threads = 8)
DefaultAssay(combined_sct)  <- "integrated"

In [ ]:
DEG_Libra_edger_lrt_cell_assigned <- DEG_Libra_edger_lrt_cell_assigned %>% group_by(cell_type) %>% arrange(desc(abs(avg_logFC)),.by_group = TRUE, p_val_adj)
DEG_Libra_edger_lrt_cell_assigned%>% head
write.csv(DEG_Libra_edger_lrt_cell_assigned, "results_seurat/cell_assigned/mut_vs_sib_libra_EdgeR_Lrt_cell_assigned.csv")

In [ ]:
# plot heatmaps for significant genes for each cell type with padj < 0.05
cell_type <- DEG_Libra_edger_lrt_cell_assigned$cell_type %>% unique %>% as.character %>% sort
cell_type

for (i in 1:length(cell_type)){
    
genes_to_plot <- DEG_Libra_edger_lrt_cell_assigned$gene[DEG_Libra_edger_lrt_cell_assigned$p_val_adj<=0.05 & DEG_Libra_edger_lrt_cell_assigned$cell_type==cell_type[i]]
print(paste0("genes significant in ",cell_type[i], " are:"))
genes_to_plot %>% print

if(length(genes_to_plot)==0){
    print(paste0("no significant genes in cluster ", cell_type[i]))
    next
}

heatmaps <- DoHeatmap(subset(combined_sct, idents = cell_type[i]),features = genes_to_plot,group.by= "condition", assay = "RNA",slot = "counts" )+ ggtitle(paste0(cell_type[i], " significant genes"))
print(heatmaps)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/heatmaps_significant_genes_mut_vs_sib/",cell_type[i],"_heatmap_significant_genes_mut_vs_sib_edgerlrt_cell_assigned.pdf"),
    width = 10,
    height = 10
    )
    dev.off ()
}

In [ ]:
# plot heatmaps for significant genes for each cell type with pval<=0.05
cell_type <- DEG_Libra_edger_lrt_cell_assigned$cell_type %>% unique %>% as.character %>% sort
cell_type

for (i in 1:length(cell_type)){
    
genes_to_plot <- DEG_Libra_edger_lrt_cell_assigned$gene[DEG_Libra_edger_lrt_cell_assigned$p_val<=0.05 & DEG_Libra_edger_lrt_cell_assigned$cell_type==cell_type[i]]
print(paste0("genes significant (", length(genes_to_plot),") in ",cell_type[i], " are (printing top 20):" ))
genes_to_plot %>%  head(20) %>% print

if(length(genes_to_plot)==0){
    print(paste0("no significant genes in cluster ", cell_type[i]))
    next
}

heatmaps <- DoHeatmap(subset(combined_sct, idents = cell_type[i]),features = genes_to_plot,group.by= "condition", assay = "RNA",slot = "counts" )+ ggtitle(paste0(cell_type[i], " significant genes"))
print(heatmaps)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/heatmaps_significant_genes_mut_vs_sib_pva05/",cell_type[i],"_heatmap_significant_genes_mut_vs_sib_edgerlrt_pval05_cell_assigned.pdf"),
    width = 10,
    height = 10
    )
    dev.off ()
}

In [ ]:
# merge with Mouse genes
DEG_Libra_edger_lrt_cell_assigned_mouse <- merge(DEG_Libra_edger_lrt_cell_assigned, Mouse_Genes, by.x = "gene", by.y = "ZFIN.symbol", all.x = TRUE)
DEG_Libra_edger_lrt_cell_assigned_mouse  <- DEG_Libra_edger_lrt_cell_assigned_mouse %>% arrange(cell_type, p_val_adj, desc(abs(avg_logFC)))
DEG_Libra_edger_lrt_cell_assigned_mouse %>% write.csv("results_seurat/cell_assigned/mut_vs_sib_libra_EdgeR_Lrt_cell_assigned_mouse.csv")

DEG_Libra_edger_lrt_cell_assigned_mouse <- merge(DEG_Libra_edger_lrt_cell_assigned, Mouse_Genes[!duplicated(Mouse_Genes$ZFIN.symbol),], by.x = "gene", by.y = "ZFIN.symbol", all.x = TRUE)
DEG_Libra_edger_lrt_cell_assigned_mouse  <- DEG_Libra_edger_lrt_cell_assigned_mouse %>% arrange(cell_type, p_val_adj, desc(abs(avg_logFC)))
DEG_Libra_edger_lrt_cell_assigned_mouse %>% head

# Number of cells in each cluster

In [ ]:
combined_sct@meta.data$condition <- NULL
combined_sct@meta.data$condition[grepl("sib",combined_sct@meta.data$orig.ident)] <- "sib"
combined_sct@meta.data$condition[grepl("mut",combined_sct@meta.data$orig.ident)] <- "mut"
combined_sct@meta.data$condition %>% unique()

In [ ]:
table(combined_sct@meta.data$cell_type, combined_sct@meta.data$orig.ident)
table(combined_sct@meta.data$cell_type, combined_sct@meta.data$condition)

In [ ]:
cell_type_numbers <- table(combined_sct@meta.data$cell_type_EC, combined_sct@meta.data$orig.ident)
cell_type_numbers <- as.data.frame(cell_type_numbers)
cell_type_numbers %>% head
cell_type_numbers <- cell_type_numbers %>% dplyr::rename(cell_type = Var1, sample_name = Var2)
cell_type_numbers$sample_name <- factor(cell_type_numbers$sample_name, levels = c("sib_1","sib_2", "mut_1", "mut_2"))
cell_type_numbers %>% write.csv("results_seurat/cell_type_numbers.csv")
cell_type_numbers %>% head

In [ ]:
# plot these percentages using ggplot2
# save the graph as a pdf file
# dev.copy(pdf, "results_seurat/cell_type_percentages_normalized.pdf", width = 10, height = 10)
ggplot(cell_type_percentages, aes(x = cell_type, y = Freq, fill = sample_name)) +
  geom_bar(stat = "identity", position = "fill") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) +
  labs(x = "Cell_type", y = "Percentage of cells", fill = "Sample name")
# dev.off()

In [ ]:
# calculate the percentage of cell in each cell type according to condition
cell_type_percentages <- prop.table(table(combined_sct@meta.data$cell_type_EC, combined_sct@meta.data$orig.ident))*100
cell_type_percentages <- as.data.frame(cell_type_percentages)
cell_type_percentages %>% head
cell_type_percentages <- cell_type_percentages %>% dplyr::rename(cell_type = Var1, sample_name = Var2)
cell_type_percentages$sample_name <- factor(cell_type_percentages$sample_name, levels = c("sib_1","sib_2", "mut_1", "mut_2"))
cell_type_percentages %>% write.csv("results_seurat/cell_type_percentages.csv")
cell_type_percentages %>% head

# plot these percentages using ggplot2
# save the graph as a pdf file
dev.copy(pdf, "results_seurat/cell_type_percentages_normalized.pdf", width = 10, height = 10)
ggplot(cell_type_percentages, aes(x = cell_type, y = Freq, fill = sample_name)) +
  geom_bar(stat = "identity", position = "fill") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) +
  labs(x = "Cell_type", y = "Percentage of cells", fill = "Sample name")
dev.off()


In [ ]:
# calculate the percentage of cell in each cell type according to condition
# save the graph as a pdf file
dev.copy(pdf, "results_seurat/cell_type_percentages.pdf", width = 10, height = 10)
ggplot(cell_type_percentages, aes(x = cell_type, y = Freq, fill = sample_name)) +
  geom_bar(stat = "identity", position = "dodge") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) +
  labs(x = "Cell_type", y = "Percentage of cells", fill = "Sample name")
dev.off()

In [ ]:
cell_type_cond_numbers <- table(combined_sct@meta.data$cell_type_EC, combined_sct@meta.data$condition)
cell_type_cond_numbers <- as.data.frame(cell_type_cond_numbers)
cell_type_cond_numbers <- cell_type_cond_numbers %>% dplyr::rename(cell_type = Var1, sample_name = Var2)
cell_type_cond_numbers$sample_name <- factor(cell_type_cond_numbers$sample_name, levels = c("sib","mut"))
cell_type_cond_numbers %>% write.csv("results_seurat/cell_type_cond_numbers.csv")
cell_type_cond_numbers

In [ ]:
# calculate the percentage of cell in each cell type according to condition
cell_type_percentages_cond <- prop.table(table(combined_sct@meta.data$cell_type_EC, combined_sct@meta.data$condition))*100
cell_type_percentages_cond <- as.data.frame(cell_type_percentages_cond)
cell_type_percentages_cond %>% head
cell_type_percentages_cond <- cell_type_percentages_cond %>% dplyr::rename(cell_type = Var1, condition = Var2)
cell_type_percentages_cond$condition <- factor(cell_type_percentages_cond$condition, levels = c("sib","mut"))
cell_type_percentages_cond %>% write.csv("results_seurat/cell_type_percentages_cond.csv")
cell_type_percentages_cond %>% head
# plot these percentages using ggplot2
# save the graph as a pdf file
dev.copy(pdf, "results_seurat/cell_type_percentages_cond_normalized.pdf", width = 10, height = 10)
ggplot(cell_type_percentages_cond, aes(x = cell_type, y = Freq, fill = condition)) +
  geom_bar(stat = "identity", position = "fill") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.5)) +
  labs(x = "Condition", y = "Percentage of cells", fill = "Cell type")
  dev.off()

In [ ]:
# calculate the percentage of cell in each cell type according to condition
# plot these percentages using ggplot2
# save the graph as a pdf file
dev.copy(pdf, "results_seurat/cell_type_percentages_cond.pdf", width = 10, height = 10)
ggplot(cell_type_percentages_cond, aes(x = cell_type, y = Freq, fill = condition)) +
  geom_bar(stat = "identity", position = "dodge") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.5)) +
  labs(x = "Condition", y = "Percentage of cells", fill = "Cell type")
  dev.off()

# Pathway analysis for Differential Genes in each cluster

In [ ]:
cell_type <- DEG_Libra_edger_lrt_cell_assigned$cell_type %>% unique %>% as.character %>% sort
cell_type

In [ ]:
DEG_Libra_edger_lrt_cell_assigned_mouse <- merge(DEG_Libra_edger_lrt_cell_assigned, Mouse_Genes[!duplicated(Mouse_Genes$ZFIN.symbol),], by.x = "gene", by.y = "ZFIN.symbol", all.x = TRUE)
DEG_Libra_edger_lrt_cell_assigned_mouse  <- DEG_Libra_edger_lrt_cell_assigned_mouse %>% arrange(cell_type, p_val_adj, desc(abs(avg_logFC)))
DEG_Libra_edger_lrt_cell_assigned_mouse %>% head

In [ ]:
cell_type <- DEG_Libra_edger_lrt_cell_assigned$cell_type %>% unique %>% as.character %>% sort
cell_type

## Over representation Analysis

In [ ]:
pval_enrich <- 0.2
cell_type <- DEG_Libra_edger_lrt_cell_assigned$cell_type %>% unique %>% as.character %>% sort
cell_type
# Loop through each cell type
for(i in cell_type){
    # Print message indicating which cell type is being analyzed
    message(paste0("Molecular function Enrichments for Cell type: ", i))
    
    # Perform GO enrichment analysis for DEG for the current cell type
    compGO_MF_diff <- enrichGO(gene = DEG_Libra_edger_lrt_cell_assigned_mouse$MGI.symbol[DEG_Libra_edger_lrt_cell_assigned_mouse$cell_type==i & DEG_Libra_edger_lrt_cell_assigned_mouse$p_val<=0.05], pvalueCutoff  = pval_enrich,keyType = "SYMBOL",
                             pAdjustMethod = "BH",OrgDb = "org.Mm.eg.db", ont = "MF")
    
    # If no enriched pathways are found, print message and move on to next cell type
    if(is.null(compGO_MF_diff)){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    if(sum(compGO_MF_diff@result$p.adjust<pval_enrich)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    
    # Generate dotplot of enriched pathways for the current cell type
    print(dotplot(compGO_MF_diff, showCategory = 15, title = paste0("GO Pathway Enrichment Analysis for DEG \n Molecular Functions for ",i, " Cells"), 
            font.size = 12)+ scale_colour_viridis(option = "plasma", direction = 1))
    
    # Save dotplot as pdf file
    dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/pathways_DEG_pval05/Molecular_function/",i,"_GO_MF_pathways_pval05_plasma.pdf"),
    width = 10,
    height = 8
    )
    dev.off ()
    
    # Convert enriched pathways data to data frame and calculate decimal ratios
    compGO_MF_diff_df <- as.data.frame(compGO_MF_diff)
    compGO_MF_diff_df$GeneRatio_decimal <- compGO_MF_diff_df$GeneRatio
    compGO_MF_diff_df$GeneRatio_decimal <- sapply(compGO_MF_diff_df$GeneRatio_decimal, 
                                                  function(x) (eval(parse(text = as.character(x)))))
    compGO_MF_diff_df$BgRatio_decimal <- compGO_MF_diff_df$BgRatio
    compGO_MF_diff_df$BgRatio_decimal <- sapply(compGO_MF_diff_df$BgRatio_decimal, 
                                                function(x) (eval(parse(text = as.character(x)))))
    compGO_MF_diff_df <- compGO_MF_diff_df %>% tidyr::separate_rows(geneID, sep = "/", convert = FALSE) %>%
      arrange(desc(GeneRatio_decimal))
    
    # Print the first few rows of the enriched pathways data frame
    compGO_MF_diff_df %>% head 
    
    # Save enriched pathways data frame as CSV file
    write.csv(compGO_MF_diff_df, paste0("results_seurat/cell_assigned/pathways_DEG_pval05/Molecular_function/",i,"_GO_MF_pathways_pval05.csv"))
    
    # Print message indicating that analysis for the current cell type is complete
    message(paste0("Cell type: ", i, " done"))
    message(paste0("****************************************************************************************"))
    message(paste0("\n"))
}                                            
                                                
# Loop through each cell type
for(i in cell_type){                           
    # Print message indicating which cell type is being processed
    message(paste0("Cellular component Enrichments for Cell type: ", i))
    
    # Perform GO enrichment analysis for DEG for cellular components
    compGO_CC_diff <- enrichGO(gene = DEG_Libra_edger_lrt_cell_assigned_mouse$MGI.symbol[DEG_Libra_edger_lrt_cell_assigned_mouse$cell_type==i & DEG_Libra_edger_lrt_cell_assigned_mouse$p_val<=0.05], 
                               pvalueCutoff  = pval_enrich,
                               keyType = "SYMBOL",
                               pAdjustMethod = "BH",
                               OrgDb = "org.Mm.eg.db",
                               ont = "CC")
    
    # If there are no enriched pathways, skip to the next cell type
    if(is.null(compGO_CC_diff)){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    if(sum(compGO_CC_diff@result$p.adjust<pval_enrich)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    # Generate a dotplot of enriched pathways for cellular components
    print(dotplot(compGO_CC_diff, 
                  showCategory = 15, 
                  title = paste0("GO Pathway Enrichment Analysis for DEG \n Cellular components for ",i, " Cells"), 
                  font.size = 12)+ scale_colour_viridis(option = "plasma", direction = 1))
    
    # Save the dotplot as an pdf file
    # dev.new()
    dev.copy(
        pdf,
        file = paste0("results_seurat/cell_assigned/pathways_DEG_pval05/Cellular_component/",i,"_GO_CC_pathways_pval05_plasma.pdf"),
        width = 10,
        height = 8
    )
    dev.off ()
    
    # Convert the enriched pathways data to a data frame
    compGO_CC_diff_df <- as.data.frame(compGO_CC_diff)
    
    # Convert GeneRatio and BgRatio to decimal format
    compGO_CC_diff_df$GeneRatio_decimal <- compGO_CC_diff_df$GeneRatio
    compGO_CC_diff_df$GeneRatio_decimal <- sapply(compGO_CC_diff_df$GeneRatio_decimal, 
                                                  function(x) (eval(parse(text = as.character(x)))))
    compGO_CC_diff_df$BgRatio_decimal <- compGO_CC_diff_df$BgRatio
    compGO_CC_diff_df$BgRatio_decimal <- sapply(compGO_CC_diff_df$BgRatio_decimal, 
                                                function(x) (eval(parse(text = as.character(x)))))
    
    # Separate gene IDs and arrange by GeneRatio_decimal in descending order
    compGO_CC_diff_df <- compGO_CC_diff_df %>% 
        tidyr::separate_rows(geneID, sep = "/", convert = FALSE) %>%
        arrange(desc(GeneRatio_decimal))
    
    # Print the first few rows of the enriched pathways data frame
    compGO_CC_diff_df %>% head
    
    # Save the enriched pathways data frame as a CSV file
    write.csv(compGO_CC_diff_df, paste0("results_seurat/cell_assigned/pathways_DEG_pval05/Cellular_component/",i,"_GO_CC_pathways_pval05.csv"))  
    
    # Print message indicating that the cell type is done processing
    message(paste0("Cell type: ", i, " done"))
    message(paste0("****************************************************************************************"))
    message(paste0("\n"))
}                                                                                    
    
                                                
# Loop through each cell type
for(i in cell_type){
    # Print message indicating which cell type is being processed
    message(paste0("Biological pathways Enrichments for Cell type: ", i))
    
    # Perform GO enrichment analysis for DEG for biological pathways
    compGO_BP_diff <- enrichGO(gene = DEG_Libra_edger_lrt_cell_assigned_mouse$MGI.symbol[DEG_Libra_edger_lrt_cell_assigned_mouse$cell_type==i & DEG_Libra_edger_lrt_cell_assigned_mouse$p_val<=0.05], pvalueCutoff  = pval_enrich,keyType = "SYMBOL",
                             pAdjustMethod = "BH",OrgDb = "org.Mm.eg.db", ont = "BP")
    
    # If there are no enriched pathways, skip to the next cell type
    if(is.null(compGO_BP_diff)){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    if(sum(compGO_BP_diff@result$p.adjust<pval_enrich)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    # Create a dotplot of the enriched pathways
    print(dotplot(compGO_BP_diff, showCategory = 15, title = paste0("GO Pathway Enrichment Analysis for DEG \n Biological pathways for ",i, " Cells"), 
            font.size = 12)+ scale_colour_viridis(option = "plasma", direction = 1))
    
    # Save the dotplot as an pdf file
    dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/pathways_DEG_pval05/Biological_pathways/",i,"_GO_BP_pathways_pval05_plasma.pdf"),
    width = 10,
    height = 8
    )
    dev.off ()
    
    # Convert the enriched pathways data to a dataframe and calculate decimal ratios
    compGO_BP_diff_df <- as.data.frame(compGO_BP_diff)
    compGO_BP_diff_df$GeneRatio_decimal <- compGO_BP_diff_df$GeneRatio
    compGO_BP_diff_df$GeneRatio_decimal <- sapply(compGO_BP_diff_df$GeneRatio_decimal, 
                                                  function(x) (eval(parse(text = as.character(x)))))
    compGO_BP_diff_df$BgRatio_decimal <- compGO_BP_diff_df$BgRatio
    compGO_BP_diff_df$BgRatio_decimal <- sapply(compGO_BP_diff_df$BgRatio_decimal, 
                                                function(x) (eval(parse(text = as.character(x)))))
    
    # Separate the gene IDs into separate rows and sort by descending GeneRatio_decimal
    compGO_BP_diff_df <- compGO_BP_diff_df %>% tidyr::separate_rows(geneID, sep = "/", convert = FALSE) %>%
      arrange(desc(GeneRatio_decimal))
    
    # Print the first few rows of the enriched pathways dataframe
    compGO_BP_diff_df %>% head
    
    # Save the enriched pathways dataframe as a CSV file
    write.csv(compGO_BP_diff_df, paste0("results_seurat/cell_assigned/pathways_DEG_pval05/Biological_pathways/",i,"_GO_BP_pathways_pval05.csv")) 
    
    # Print message indicating that the cell type is done processing
    message(paste0("Cell type: ", i, " done"))
    message(paste0("****************************************************************************************"))
    message(paste0("\n"))
}

## Geneset Enrichment Analysis

In [ ]:
cell_type <- DEG_Libra_edger_lrt_cell_assigned$cell_type %>% unique %>% as.character %>% sort
cell_type

In [ ]:
pval_enrich <- 0.2
cell_type <- DEG_Libra_edger_lrt_cell_assigned$cell_type %>% unique %>% as.character %>% sort
cell_type
# Loop through each cell type
for(i in cell_type){
    # Print message indicating which cell type is being analyzed
    message(paste0("Molecular function Enrichments for Cell type: ", i))
        
    # create gene list for the current cell type
    gene_list_df <- DEG_Libra_edger_lrt_cell_assigned_mouse[DEG_Libra_edger_lrt_cell_assigned_mouse$cell_type==i & DEG_Libra_edger_lrt_cell_assigned_mouse$p_val<=0.05,]
    gene_list_df <- gene_list_df %>% arrange(desc(avg_logFC))
    gene_list_df <- gene_list_df[!is.na(gene_list_df$MGI.symbol),]
    gene_list_df <- gene_list_df[!duplicated(gene_list_df$MGI.symbol),]
    gene_list <- gene_list_df %>% pull(avg_logFC)
    names(gene_list) <- gene_list_df %>% pull(MGI.symbol)

    gene_list <- gene_list[!duplicated(gene_list)]
     if(is.null(gene_list)|length(gene_list)==0){
      
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    # Perform GO enrichment analysis for DEG for the current cell type
    compGO_MF_diff <- gseGO(gene = gene_list, pvalueCutoff  = pval_enrich,keyType = "SYMBOL",
                             pAdjustMethod = "BH",OrgDb = "org.Mm.eg.db", ont = "MF")
    # If no enriched pathways are found, print message and move on to next cell type
    if(is.null(compGO_MF_diff)|nrow(compGO_MF_diff@result)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    if(sum(compGO_MF_diff@result$p.adjust<pval_enrich)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    # Generate dotplot of enriched pathways for the current cell type
   print(dotplot(compGO_MF_diff, showCategory = 15, title = paste0("GO Pathway Geneset Enrichment Analysis for DEG \n Molecular Functions for ",i, " Cells"), 
            font.size = 12) + facet_grid(.~.sign)+ scale_colour_viridis(option = "plasma", direction = 1))
    # Save dotplot as pdf file
    dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/pathways_DEG_pval05_GSE/Molecular_function/",i,"_gse_GO_MF_pathways_pval05_plasma.pdf"),
    width = 10,
    height = 8
    )
    dev.off ()
    
    # Convert enriched pathways data to data frame and calculate decimal ratios
    # Generate dotplot of enriched pathways for the current cell type
  
    
        compGO_MF_diff_df <- as.data.frame(compGO_MF_diff)
        compGO_MF_diff_df

    compGO_MF_diff_df <- compGO_MF_diff_df %>% tidyr::separate_rows(core_enrichment, sep = "/", convert = FALSE) %>%
      arrange((p.adjust))
    
    # Save enriched pathways data frame as CSV file
    write.csv(compGO_MF_diff_df, paste0("results_seurat/cell_assigned/pathways_DEG_pval05_GSE/Molecular_function/",i,"_gse_GO_MF_pathways_pval05.csv"))
    
    # Print message indicating that analysis for the current cell type is complete
    message(paste0("Cell type: ", i, " done"))
    message(paste0("****************************************************************************************"))
    message(paste0("\n"))
}                                            
                                                
# Loop through each cell type
for(i in cell_type){                           
    # Print message indicating which cell type is being processed
    message(paste0("Cellular component Enrichments for Cell type: ", i))
    
     # create gene list for the current cell type
    gene_list_df <- DEG_Libra_edger_lrt_cell_assigned_mouse[DEG_Libra_edger_lrt_cell_assigned_mouse$cell_type==i & DEG_Libra_edger_lrt_cell_assigned_mouse$p_val<=0.05,]
    gene_list_df <- gene_list_df %>% arrange(desc(avg_logFC))
    gene_list_df <- gene_list_df[!is.na(gene_list_df$MGI.symbol),]
    gene_list_df <- gene_list_df[!duplicated(gene_list_df$MGI.symbol),]
    gene_list <- gene_list_df %>% pull(avg_logFC)
    names(gene_list) <- gene_list_df %>% pull(MGI.symbol)

    gene_list <- gene_list[!duplicated(gene_list)]


    gene_list <- gene_list[!duplicated(gene_list)]
     if(is.null(gene_list)|length(gene_list)==0){
        # print("trace_gene_if")
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    # Perform GO enrichment analysis for DEG for cellular components
    compGO_CC_diff <- gseGO(gene = gene_list, 
                               pvalueCutoff  = pval_enrich,
                               keyType = "SYMBOL",
                               pAdjustMethod = "BH",
                               OrgDb = "org.Mm.eg.db",
                               ont = "CC")
    
    # If there are no enriched pathways, skip to the next cell type
    if(is.null(compGO_CC_diff)|nrow(compGO_CC_diff@result)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    if(sum(compGO_CC_diff@result$p.adjust<pval_enrich)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    # Generate a dotplot of enriched pathways for cellular components
    print(dotplot(compGO_CC_diff, 
                  showCategory = 15, 
                  title = paste0("GO Pathway Geneset Enrichment Analysis for DEG \n Cellular components for ",i, " Cells"), 
                  font.size = 12)+ facet_grid(.~.sign)+ scale_colour_viridis(option = "plasma", direction = 1))
    
    # Save the dotplot as an pdf file

    dev.copy(
        pdf,
        file = paste0("results_seurat/cell_assigned/pathways_DEG_pval05_GSE/Cellular_component/",i,"_gse_GO_CC_pathways_pval05_plasma.pdf"),
        width = 10,
        height = 8
    )
    dev.off ()
    
    # Convert the enriched pathways data to a data frame
    compGO_CC_diff_df <- as.data.frame(compGO_CC_diff)
    compGO_CC_diff_df <- compGO_CC_diff_df %>% 
        tidyr::separate_rows(core_enrichment, sep = "/", convert = FALSE) %>%
        arrange(p.adjust)
    
    # Print the first few rows of the enriched pathways data frame
    compGO_CC_diff_df %>% head
    
    # Save the enriched pathways data frame as a CSV file
    write.csv(compGO_CC_diff_df, paste0("results_seurat/cell_assigned/pathways_DEG_pval05_GSE/Cellular_component/",i,"_gse_GO_CC_pathways_pval05.csv"))  
    
    # Print message indicating that the cell type is done processing
    message(paste0("Cell type: ", i, " done"))
    message(paste0("****************************************************************************************"))
    message(paste0("\n"))
}                                                                                    
    
                                                
# Loop through each cell type
for(i in cell_type){
    # Print message indicating which cell type is being processed
    message(paste0("Biological pathways Enrichments for Cell type: ", i))
    
    gene_list_df <- DEG_Libra_edger_lrt_cell_assigned_mouse[DEG_Libra_edger_lrt_cell_assigned_mouse$cell_type==i & DEG_Libra_edger_lrt_cell_assigned_mouse$p_val<=0.05,]
    gene_list_df <- gene_list_df %>% arrange(desc(avg_logFC))
    gene_list_df <- gene_list_df[!is.na(gene_list_df$MGI.symbol),]
    gene_list_df <- gene_list_df[!duplicated(gene_list_df$MGI.symbol),]
    gene_list <- gene_list_df %>% pull(avg_logFC)
    names(gene_list) <- gene_list_df %>% pull(MGI.symbol)

    gene_list <- gene_list[!duplicated(gene_list)]
    gene_list %>% head
    gene_list %>% length
    gene_list_df %>% head()


    gene_list <- gene_list[!duplicated(gene_list)]
     if(is.null(gene_list)|length(gene_list)==0){
        # print("trace_gene_if")
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    # Perform GO enrichment analysis for DEG for biological pathways
    compGO_BP_diff <- gseGO(gene = gene_list, pvalueCutoff  = pval_enrich,keyType = "SYMBOL",
                             pAdjustMethod = "BH",OrgDb = "org.Mm.eg.db", ont = "BP")
    
    # If there are no enriched pathways, skip to the next cell type
    if(is.null(compGO_BP_diff)|nrow(compGO_BP_diff@result)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    if(sum(compGO_BP_diff@result$p.adjust<pval_enrich)==0){
        message(paste0("Cell type: ", i, " done"))
        message(paste0("****************************************************************************************"))
        message(paste0("\n"))
        next
    }
    # Create a dotplot of the enriched pathways
    print(dotplot(compGO_BP_diff, showCategory = 15, title = paste0("GO Pathway Geneset Enrichment Analysis for DEG \n Biological pathways for ",i, " Cells"), 
            font.size = 12)+ facet_grid(.~.sign)+ scale_colour_viridis(option = "plasma", direction = 1))
    
    # Save the dotplot as an pdf file
    dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/pathways_DEG_pval05_GSE/Biological_pathways/",i,"_gse_GO_BP_pathways_pval05_plasma.pdf"),
    width = 10,
    height = 8
    )
    dev.off ()
    
    # Convert the enriched pathways data to a dataframe and calculate decimal ratios
    compGO_BP_diff_df <- as.data.frame(compGO_BP_diff)
    
    # Separate the gene IDs into separate rows and sort by descending GeneRatio_decimal
    compGO_BP_diff_df <- compGO_BP_diff_df %>% tidyr::separate_rows(core_enrichment, sep = "/", convert = FALSE) %>%
      arrange(p.adjust)
    
    # Print /the first few rows of the enriched pathways dataframe
    compGO_BP_diff_df %>% head
    
    # Save the enriched pathways dataframe as a CSV file
    write.csv(compGO_BP_diff_df, paste0("results_seurat/cell_assigned/pathways_DEG_pval05_GSE/Biological_pathways/",i,"_gse_GO_BP_pathways_pval05.csv")) 
    
    # Print message indicating that the cell type is done processing
    message(paste0("Cell type: ", i, " done"))
    message(paste0("****************************************************************************************"))
    message(paste0("\n"))
}

# Graph for Number of DEGs

In [ ]:
broad_sub_cluster_names <- combined_sct@meta.data %>% dplyr::select(cell_type_EC, broad_class) %>% distinct() %>% arrange(cell_type_EC)
rownames(broad_sub_cluster_names) <- NULL
broad_sub_cluster_names

In [ ]:
# filter the data for signicant values with padj <= 0.05
DEG_Libra_edger_lrt_cell_assigned_sig_padj05 <- DEG_Libra_edger_lrt_cell_assigned %>% filter(p_val_adj<=0.05)
# count how many genes are significant for each cell type
num_of_DEG_padj05 <- DEG_Libra_edger_lrt_cell_assigned_sig_padj05 %>% group_by(cell_type) %>% summarise(n=n()) %>% arrange(desc(n))
num_of_DEG_padj05

In [ ]:
# filter the data for signicant values with padj <= 0.05
DEG_Libra_edger_lrt_cell_assigned_sig_pval05 <- DEG_Libra_edger_lrt_cell_assigned %>% filter(p_val<=0.05)
# count how many genes are significant for each cell type
num_of_DEG_pval05 <- DEG_Libra_edger_lrt_cell_assigned_sig_pval05 %>% group_by(cell_type) %>% summarise(n=n()) %>% arrange(desc(n))
num_of_DEG_pval05

In [ ]:

num_of_DEG_pval05 <-merge(num_of_DEG_pval05, broad_sub_cluster_names, by.x = "cell_type", by.y = "cell_type_EC", all.x = TRUE)
head(num_of_DEG_pval05)


In [ ]:
num_of_DEG_pval05 <- num_of_DEG_pval05 %>% group_by(broad_class) %>% arrange(desc(n),cell_type,broad_class, .by_group = TRUE) 
num_of_DEG_pval05

In [ ]:
# plot a bar graph coloring the bars by broad_class arrange descending in each group
num_of_DEG_pval05 %>% group_by(broad_class) %>%  arrange(desc(n),cell_type,broad_class, .by_group = TRUE)  %>% ggplot()+aes(x=cell_type, y=n, fill=broad_class) + geom_bar(stat="identity") + theme(axis.text.x = element_text(angle = 90, hjust = 1)) + labs(title = "Number of DEG with pval <= 0.05", x = "Cell type", y = "Number of DEG")


# Heatmaps for all OFT and AV genes

In [129]:
OFT_related_genes_all <- c("axin2", "acvr1l", "atf2", "bmp4", "bmp7a", "bmp7b", "bmpr1aa", "bmpr1ab", "bmpr2a", "bmpr2b", "cdc42", "cited2", "crkl", "ctnnb1", "dhrs3a", "dhrs3b", "dicer1", "dmgdh",  "doc2b", "dpt", "dvl1a", "dvl1b", "dvl2", "dvl3a", "dvl3b", "edn1", "ednraa", "elnb", "engase", "enpp2", "eya1", "fgf10a", "fgf8a", "fgf8b", "fgfr1a", "fn1a", "foxc1a", "foxc1b", "foxh1", "fpgs", "fzd1", "fzd2", "gata4", "gata6", "gja5a", "gja5b", "hand2", "her6", "her9", "hey1", "hey2", "heyl", "hmgcrb", "ilk", "isl1", "isl1a", "jag1a", "jag1b", "jun", "klf2a", "klf2b", "lef1", "lrp2a", "lrp6", "mef2ca", "mylka", "ndr1", "nipbla", "nipblb", "nkd1", "nkx2.5", "nog1", "nog2", "nog3", "nos2a", "nos2b", "nostrin", "notch1a", "notch1b", "notum1a", "notum1b", "npy1r", "npy2r", "npy5r", "nrp1a", "nrp1b", "nrp2a", "nrp2b", "parvaa", "piezo1", "piezo2a.2", "pitx2", "pkd2", "plek", "plxnd1", "postnb", "rarb", "robo1", "robo2", "ryr1a", "ryr1b", "sec24b", "sema3c", "sfrp2", "shha", "shhb", "six1a", "six1b", "smad1", "smad4a", "smad4b", "smad5", "smad6a", "smad6b", "smad9", "smarca4a", "smarca4b", "sox11a", "sox11b", "sox17", "sox32", "tbx1", "tbx20", "tbx2a", "tbx2b", "tbx3a", "tbx3b", "tbx5a", "tbx5b", "tfap2a", "tgfb1a", "tgfb2", "tgfb3", "tgfbr2a", "tgfbr2b", "tgfbr2l", "tgfbr3", "thbs1a", "thbs1b", "trpv4", "twist1a", "twist1b", "vangl2", "wnt11", "wnt11f2", "wnt16", "wnt5a", "wnt9b", "zfpm1", "zfpm2a", "zfpm2b") 

In [ ]:
# change graph size
options(repr.plot.width=24, repr.plot.height=24)
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'EnC_1', 'EnC_2', 'EnC_3', 'EnC_4', 'EnC_Valve_cells' ))), isGene(OFT_related_genes_all,combined_sct, return.values = TRUE),
    annot.by = c("cell_type_EC", "condition"), complex = TRUE,
    heatmap.colors = PurpleAndYellow(50),
    # scaled.to.max=TRUE,
    # scale='none',
    heatmap.colors.max.scaled = PurpleAndYellow(50))

In [95]:
AV_related_genes_all <- c("acvr1l", "adamts5", "anxa5b", "apcdd1l", "aplnra", "aplnrb", "axin2", "bgna", "bmp2a", "bmp2b", "bmp4", "bmpr1aa", "bmpr1ab", "bmpr2a", "bmpr2b", "c7a", "c7b", "ccdc80", "ccn1", "cldn5a", "cldn5b", "clu", "col1a1a", "col1a1b", "col1a2", "col2a1a", "col9a2", "crabp2a", "crabp2b", "cthrc1a", "cthrc1b", "dchs1b", "dcn", "dkk3a", "dkk3b", "dll4", "dmgdh", "doc2b", "dpt", "dusp2", "efna1a", "efna1b", "egfl7", "elnb", "emilin1a", "emilin1b", "eng", "enpp2", "esama", "esamb", "fbln1", "fbln2", "flna", "flrt2", "fmoda", "fmodb", "fstl1a", "gata3", "gata4", "gata5", "gja5a", "gja5b", "gpc3", "hapln1a", "hapln1b", "has2", "hey1", "hey2", "heyl", "hspg2", "id2a", "id2b", "id3", "id4", "igfbp7", "ipo11", "itga4", "itga5", "jag1a", "jag1b", "junba", "klf2a", "klf2b", "lef1", "limch1a", "limch1b", "lmna", "loxl2a", "loxl2b", "lum", "mdm2", "mdm4", "mxra5a", "mxra8a", "mxra8b", "myh10", "naglu", "ncam1a", "ncam1b", "nfatc1", "nkd1", "nos3", "notch1a", "notch1b", "notum1a", "notum1b", "nrg1", "nrg2a", "nrg2b", "ociad2", "pam", "pdgfaa", "pdlim3a", "pdlim3b", "piezo1", "pitx2", "plek", "postnb", "prc1a", "prc1b", "prrx1a", "prrx1b", "rb1", "rbp1.1", "rbpja", "rbpjb", "rnd2", "robo1", "robo2", "s100a11", "samd11", "scinla", "serpine2", "serpinf1", "sgk1", "slit2", "slit3", "smad2", "smad4a", "smad4b", "smad6a", "smad6b", "snai1a", "snai1b", "sost", "sox4a", "sox9a", "sox9b", "sparc", "sparcl1", "spp1", "tbx20", "tbx2a", "tbx2b", "tbx3a", "tbx3b", "tbx5a", "tbx5b", "tgfb1a", "tgfb1b", "tgfb2", "tgfbi", "tgfbr2a", "tgfbr2b", "tgm2a", "tgm2b", "thbs1a", "tie1", "tnfrsf1a", "tnfrsf1b", "twist1a", "twist1b", "upp1", "vcana", "vcanb", "wnt16", "wnt4", "wnt9b", "zbtb20", "zfpm1", "zfpm2a", "zfpm2b")

In [ ]:
# change graph size
options(repr.plot.width=24, repr.plot.height=24)
DoHeatmap(combined_sct, features= AV_related_genes_all) + NoLegend() + ggtitle("AV related genes")
DoHeatmap(combined_sct, features= AV_related_genes_all,assay = "RNA",slot = "counts" ) + ggtitle("AV related genes") + NoLegend()

In [ ]:
# change graph size
options(repr.plot.width=24, repr.plot.height=24)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/OFT_related_genes_in_Endocardial_cells_normalized_data.pdf"),
    width = 10,
    height = 18
    )
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'EnC_1', 'EnC_2', 'EnC_3', 'EnC_4', 'EnC_Valve_cells' ))), isGene(OFT_related_genes_all,combined_sct, return.values = TRUE),
    annot.by = c("cell_type_EC", "condition"), complex = TRUE,
    heatmap.colors = PurpleAndYellow(50),
    # scaled.to.max=TRUE,
    # scale='none',
    use_raster=TRUE,
    heatmap.colors.max.scaled = PurpleAndYellow(50),
    main= "OFT_related_genes_all (normalized data)")
dev.off()

### AV in CM

In [ ]:
# change graph size
options(repr.plot.width=24, repr.plot.height=24)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/AV_related_genes_in_CM_cells_normalized_data.pdf"),
    width = 10,
    height = 18
    )
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'CM_Ventricle', 'CM_Atrium', 'CM_AV_canal' ))), isGene(AV_related_genes_all,combined_sct, return.values = TRUE),
    annot.by = c("cell_type_EC", "condition"), complex = TRUE,
    heatmap.colors = PurpleAndYellow(50),
    # scaled.to.max=TRUE,
    # scale='none',
    use_raster=TRUE,
    heatmap.colors.max.scaled = PurpleAndYellow(50),
    main= "AV_related_genes_all in CM cells (normalized data)")



dev.off()




In [ ]:
options(repr.plot.width=24, repr.plot.height=18)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/AV_related_genes_in_CM_cells_raw_counts.pdf"),
    width = 10,
    height = 18
    )
    
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'CM_Ventricle', 'CM_Atrium', 'CM_AV_canal' ))), 
            isGene(OFT_related_genes_all,combined_sct, assay = "RNA",return.values = TRUE),
            assay = "RNA",
            slot = "counts",
            annot.by = c("cell_type_EC","condition" ), complex = TRUE,
            scaled.to.max = TRUE,
            scale = 'row',
            use_raster=TRUE,
            heatmap.colors = PurpleAndYellow(50),
            heatmap.colors.max.scaled = PurpleAndYellow(25),
            main = "AV_related_genes_all in CM cells (raw counts)",
    ) %>% print
dev.off ()

In [ ]:
# change graph size
options(repr.plot.width=24, repr.plot.height=24)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/OFT_related_genes_in_BA_FB_cells_normalized_data.pdf"),
    width = 10,
    height = 18
    )
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'Mus_SMC_BA', 'FB' ))), isGene(OFT_related_genes_all,combined_sct, return.values = TRUE),
    annot.by = c("cell_type_EC", "condition"), complex = TRUE,
    heatmap.colors = PurpleAndYellow(50),
    # scaled.to.max=TRUE,
    # scale='none',
    use_raster=TRUE,
    heatmap.colors.max.scaled = PurpleAndYellow(50),
    main= "OFT_related_genes_all in BA_FB cells (normalized data)")
dev.off()

In [ ]:
options(repr.plot.width=24, repr.plot.height=18)
dev.copy(
    svg,
    file = paste0("results_seurat/cell_assigned/OFT_related_genes_in_BA_FB_cells_raw_counts.svg"),
    width = 10,
    height = 18
    )
    
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'Mus_SMC_BA', 'FB' ))), 
            isGene(OFT_related_genes_all,combined_sct, assay = "RNA",return.values = TRUE),
            assay = "RNA",
            slot = "counts",
            annot.by = c("cell_type_EC","condition" ), complex = TRUE,
            scaled.to.max = TRUE,
            scale = 'row',
            use_raster=TRUE,
            heatmap.colors = PurpleAndYellow(50),
            heatmap.colors.max.scaled = PurpleAndYellow(25),
            main = "OFT related genes in Mus_SMC_BA and FB cells (raw counts)",
    ) %>% print
dev.off ()


In [ ]:
# change graph size
options(repr.plot.width=24, repr.plot.height=24)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/OFT_related_genes_in_BA_Skeletal_muscle_cells_normalized_data.pdf"),
    width = 10,
    height = 18
    )
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'Mus_SMC_BA', 'Mus_Skeletal_muscle' ))), isGene(OFT_related_genes_all,combined_sct, return.values = TRUE),
    annot.by = c("cell_type_EC", "condition"), complex = TRUE,
    heatmap.colors = PurpleAndYellow(50),
    # scaled.to.max=TRUE,
    # scale='none',
    use_raster=TRUE,
    heatmap.colors.max.scaled = PurpleAndYellow(50),
    main= "OFT_related_genes_all in BA_Skeletal_muscle cells (normalized data)")
dev.off()

In [ ]:
options(repr.plot.width=24, repr.plot.height=18)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/OFT_related_genes_in_BA_Skeletal_muscle_cells_raw_counts.pdf"),
    width = 10,
    height = 18
    )
    
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'Mus_SMC_BA', 'Mus_Skeletal_muscle' ))), 
            isGene(OFT_related_genes_all,combined_sct, assay = "RNA",return.values = TRUE),
            assay = "RNA",
            slot = "counts",
            annot.by = c("cell_type_EC","condition" ), complex = TRUE,
            scaled.to.max = TRUE,
            scale = 'row',
            use_raster=TRUE,
            heatmap.colors = PurpleAndYellow(50),
            heatmap.colors.max.scaled = PurpleAndYellow(25),
            main = "OFT related genes in BA and Skeletal muscle cells (raw counts)",
    ) %>% print
dev.off ()

# Transfer FB subclusters to combined_sct

In [ ]:
FB_cells@meta.data$FB_identity <- paste0("FB_",(FB_cells@meta.data$SCT_snn_res.0.4)) %>% factor
FB_cells@meta.data %>% head

In [ ]:
combined_sct@meta.data$cell_name <- rownames(combined_sct@meta.data)
combined_sct@meta.data %>% head()
FB_cells@meta.data$cell_name  <- rownames(FB_cells@meta.data)
FB_cells %>% head()

In [ ]:
combined_sct@meta.data <- merge(combined_sct@meta.data, FB_cells@meta.data[,c("cell_name", "FB_identity")], by = "cell_name", all.x = TRUE) 
rownames(combined_sct@meta.data) <- combined_sct@meta.data$cell_name
combined_sct@meta.data %>% head

In [ ]:
combined_sct@meta.data$cell_type_FB <- combined_sct@meta.data$cell_type_EC
combined_sct@meta.data$FB_identity <- as.character(combined_sct@meta.data$FB_identity)
combined_sct@meta.data$cell_type_FB[!is.na(combined_sct@meta.data$FB_identity)] <- combined_sct@meta.data$FB_identity[!is.na(combined_sct@meta.data$FB_identity)]
combined_sct@meta.data$cell_type_FB <- factor(combined_sct@meta.data$cell_type_FB)
combined_sct@meta.data %>% head


In [47]:
combined_sct@meta.data$cell_type_FB  <- as.character(combined_sct@meta.data$cell_type_FB)

In [ ]:
# change graph size
options(repr.plot.width=24, repr.plot.height=24)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/AV_related_genes_in_FBs_and_unknown_2_normalized_data.pdf"),
    width = 10,
    height = 18
    )
dittoHeatmap(subset(combined_sct,subset =  (cell_type_FB %in% c( 'FB_0','FB_1','FB_2','FB_3','unknown_2' ))), isGene(AV_related_genes_all,combined_sct, return.values = TRUE),
    annot.by = c("cell_type_FB", "condition"), complex = TRUE,
    heatmap.colors = PurpleAndYellow(50),
    # scaled.to.max=TRUE,
    # scale='none',
    use_raster=TRUE,
    heatmap.colors.max.scaled = PurpleAndYellow(50),
    main= "AV_related_genes_all in FBs and unknown_2 cells (normalized data)")
dev.off()

svg 
  3

png 
  2

In [ ]:
options(repr.plot.width=24, repr.plot.height=18)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/AV_related_genes_in_FBs_and_unknown_2_raw_counts.pdf"),
    width = 10,
    height = 18
    )
    
dittoHeatmap(subset(combined_sct,subset =  (cell_type_EC %in% c( 'FB_0','FB_1','FB_2','FB_3','unknown_2' ))), 
            isGene(AV_related_genes_all,combined_sct, assay = "RNA",return.values = TRUE),
            assay = "RNA",
            slot = "counts",
            annot.by = c("cell_type_EC","condition" ), complex = TRUE,
            scaled.to.max = TRUE,
            scale = 'row',
            use_raster=TRUE,
            heatmap.colors = PurpleAndYellow(50),
            heatmap.colors.max.scaled = PurpleAndYellow(25),
            main = "AV_related_genes_all in FBs and unknown_2 cells (raw counts)",
    ) %>% print
dev.off ()

# Ligand recepetor analysis using Liana

In [ ]:
DefaultAssay(combined_sct) <- "SCT"
DefaultAssay(combined_sct)

In [ ]:
show_homologene()

In [ ]:
op_resource <- select_resource("Consensus")[[1]]

# Generate orthologous resource
ortholog_resource <- generate_homologs(op_resource = op_resource,
                                       target_organism = 7955) 

In [ ]:
liana_mut <- liana_wrap(subset(combined_sct,subset =  (condition %in% c( 'mut' ))), resource = 'custom', # resource has to be set to 'custom' to work with external resources
                        external_resource = ortholog_resource)
liana_mut %>% dplyr::glimpse()

liana_res_mut <- liana_mut %>%
  liana_aggregate()

liana_res_mut %>%
  liana_dotplot(source_groups = c("CM_Ventricle"),
                target_groups = c("EnC_Valve_cells", "CM_AV_canal"),
                ntop = 20)

liana_res_mut %>%
  liana_dotplot(source_groups = c("CM_Atrium"),
                target_groups = c("EnC_Valve_cells", "CM_AV_canal"),
                ntop = 20)

In [ ]:
liana_sib <- liana_wrap(subset(combined_sct,subset =  (condition %in% c( 'sib' ))), resource = 'custom', # resource has to be set to 'custom' to work with external resources
                        external_resource = ortholog_resource)
liana_sib %>% dplyr::glimpse()



In [ ]:
liana_res_mut_sig <- liana_res_mut %>% filter(aggregate_rank < 0.05) 
liana_res_sib_sig <- liana_res_sib %>% filter(aggregate_rank < 0.05)
liana_res_mut_sig %>% head
liana_res_sib_sig  %>% head

In [48]:
# add a row to liana_res_mut_sig with source =  CM_AV_canal and target = EnC_3 so that columns are not missing in graphs
liana_res_mut_sig_1 <- liana_res_mut_sig %>% add_row(source = "CM_AV_canal", target = "EnC_3",ligand.complex = "angpt1", receptor.complex= "itga5")%>% add_row(source = "CM_AV_canal", target = "CM_AV_canal", ligand.complex = "fstl1b", receptor.complex= "dip2a")

In [ ]:
source <- c( "CM_Ventricle", "CM_Atrium", "CM_AV_canal")
target <- c("EnC_Valve_cells", "EnC_1", "EnC_2", "EnC_3", "EnC_4")
cond <- "mut"
dev.copy(pdf, paste0("results_seurat/cell_assigned/LR_analysis/liana_", cond,"_",  paste0(source, collapse="_"), "__TO__", paste0(target,collapse = "_"),"_significant_agg05_15x15_plasma.pdf", collapse = "_"), width = 15, height = 15)
liana_res_mut_sig_1 %>%
  liana_dotplot(source_groups = source,
                target_groups = target,
                specificity = "natmi.edge_specificity",
                magnitude = "aggregate_rank",
                colour.label = "Aggregate Rank \nPval",
                ntop = 20) +
                # rotate x axis labels
                 theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))+ scale_colour_viridis(option = "plasma", direction = 1)
dev.off()

cond <- "sib"

dev.copy(pdf, paste0("results_seurat/cell_assigned/LR_analysis/liana_", cond,"_",  paste0(source, collapse="_"), "__TO__", paste0(target,collapse = "_"),"_significant_agg05_15x15_plasma.pdf", collapse = "_"), width = 15, height = 15)
liana_res_sib_sig %>%
  liana_dotplot(source_groups = source,
                target_groups = target,
                specificity = "natmi.edge_specificity",
                magnitude = "aggregate_rank",
                colour.label = "Aggregate Rank \nPval",
                ntop = 20) +
                # rotate x axis labels
                 theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))+ scale_colour_viridis(option = "plasma", direction = 1)
dev.off()

In [ ]:
source <- c("EnC_Valve_cells", "EnC_1", "EnC_2", "EnC_3", "EnC_4")
target <- c( "CM_Ventricle", "CM_Atrium", "CM_AV_canal")
cond <- "mut"
dev.copy(pdf, paste0("results_seurat/cell_assigned/LR_analysis/liana_", cond,"_",  paste0(source, collapse="_"), "__TO__", paste0(target,collapse = "_"),"_significant_agg05.pdf", collapse = "_"), width = 12, height = 15)
liana_res_mut_sig %>%
  liana_dotplot(source_groups = source,
                target_groups = target,
                specificity = "natmi.edge_specificity",
                magnitude = "aggregate_rank",
                colour.label = "Aggregate Rank /npval",
                ntop = 20) +
                # rotate x axis labels
                 theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))
dev.off()

cond <- "sib"

dev.copy(pdf, paste0("results_seurat/cell_assigned/LR_analysis/liana_", cond,"_",  paste0(source, collapse="_"), "__TO__", paste0(target,collapse = "_"),"_significant_agg05.pdf", collapse = "_"), width = 12, height = 15)
liana_res_sib_sig %>%
  liana_dotplot(source_groups = source,
                target_groups = target,
                specificity = "natmi.edge_specificity",
                magnitude = "aggregate_rank",
                colour.label = "Aggregate Rank /npval",
                ntop = 20) +
                # rotate x axis labels
                 theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))
dev.off()

In [117]:
write.csv(liana_res_mut, file = "results_seurat/cell_assigned/LR_analysis/liana_mut.csv")
write.csv(liana_res_sib, file = "results_seurat/cell_assigned/LR_analysis/liana_sib.csv")

write.csv(liana_res_mut_sig, file = "results_seurat/cell_assigned/LR_analysis/liana_mut_sig_agg05.csv")
write.csv(liana_res_sib_sig, file = "results_seurat/cell_assigned/LR_analysis/liana_sib_sig_agg05.csv")

# Other Figures for Paper

In [6]:
# in cell_type_EC change unknown_1 to “neuronal progenitors” and unkown 2 to “chondrocytes"
combined_sct@meta.data$cell_type_EC[combined_sct@meta.data$cell_type_EC== "unknown_1"] <- "Neuronal_Progenitors"
combined_sct@meta.data$cell_type_EC[combined_sct@meta.data$cell_type_EC== "unknown_2"] <- "Chondrocytes"

In [ ]:
factor(combined_sct@meta.data$cell_type_EC, levels = sort(unique(combined_sct@meta.data$cell_type_EC))) %>% head

In [8]:
combined_sct@meta.data$cell_type_EC <- factor(combined_sct@meta.data$cell_type_EC, levels = sort(unique(combined_sct@meta.data$cell_type_EC)))

In [ ]:
combined_sct@meta.data <- combined_sct@meta.data[match(names(Idents(combined_sct)),rownames(combined_sct@meta.data)),] 
combined_sct@meta.data %>% head
Idents(combined_sct)  %>% head

In [11]:
Idents(combined_sct)  <- "cell_type_EC"

In [ ]:
dir.create("results_seurat/cell_assigned/figures/", showWarnings = FALSE)
dev.copy(pdf, "results_seurat/cell_assigned/figures/umap_subclusters_25072023_nolabel.pdf", 
width = 20, height = 12)


# DimPlot(combined_sct, reduction = "umap",label = TRUE, repel = TRUE, label.size=6, pt.size=1
# ) + geom_text_repel( max.overlaps = 20)+NoLegend()

dittoDimPlot(object = combined_sct, var= "ident", reduction = "umap", do.label= FALSE, labels.repel= TRUE, labels.size = 6, labels.highlight = FALSE, do.raster= FALSE, color.panel = paletteer_d("ggthemes::Tableau_20", n=length(unique(combined_sct@meta.data$cell_type_EC))))
dev.off ()

In [43]:
Idents(combined_sct) <- as.character(combined_sct@meta.data$cell_type_FB)

In [32]:
combined_sct@meta.data$cell_type_FB <- factor(combined_sct@meta.data$cell_type_FB, levels = c(unique(sort(combined_sct@meta.data$cell_type_FB))))

## Markers for ECs

In [ ]:
EnC_cells <- subset(combined_sct, subset= (cell_type_EC%in% c("EnC_1","EnC_2", "EnC_3", "EnC_4", "EnC_Valve_cells")))
EnC_cells
EnC_cells@meta.data$cell_type_EC <- factor(EnC_cells@meta.data$cell_type_EC, levels = unique(sort(EnC_cells@meta.data$cell_type_EC)))
EnC_cells@meta.data$cell_type_EC %>% head

EnC_cells@meta.data$condition <- factor(EnC_cells@meta.data$condition, levels = c("sib", "mut"))
EnC_cells@meta.data$condition %>% head

Idents(EnC_cells)  <-  factor(EnC_cells@meta.data$cell_type_EC, levels= sort(unique(EnC_cells@meta.data$cell_type_EC), decreasing = FALSE))
Idents(EnC_cells) %>% head
# Idents(EnC_cells) %>% unique

In [ ]:
Idents(EnC_cells)  <-  factor(EnC_cells@meta.data$cell_type_EC, levels= sort(unique(EnC_cells@meta.data$cell_type_FB)))
Idents(EnC_cells) %>% head

In [ ]:
options(repr.plot.width=10, repr.plot.height=8)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/figures/EnC_cells_markers_dot_plot_magma_reordered_25072023_alphabetical_reversed.pdf"),
    width = 12,
    height = 8
    )
DotPlot(object = EnC_cells, features = c("id2b", "egfl7", "esama", "doc2b", "hapln1b", "ednraa", "zfpm1", "slit3", "spp1", "hand2", "notch1b", "fn1a", "dchs1b", "alcama", "col5a1", "col1a1a", "cdh5", "thbs1a", "has2", "klf2a", "sdc2", "postnb", "mylka", "ryr1b", "col1a1b", "tie1", "vcana", "rtn4a", "tgfb2", "elnb", "zfpm2b", "smad6b", "crip2", "col1a2", "postna", "nrp1a"), dot.min=0, dot.scale = 10, scale.min = 0, scale.max = 75)+ scale_color_viridis(option = "magma",direction = -1)+ RotatedAxis()
#cols=c(  "blue", "red"),
dev.off()

## Violin plot for notch1b

In [ ]:

options(repr.plot.width=8, repr.plot.height=10)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/figures/EnC_cells_Notch1b_dittovln_no_box_mut_sib_11102023.pdf"),
    width = 9,
    height = 6
    )
dittoPlot(object = EnC_cells, var = c("notch1b"), group.by = "condition", split.by = "cell_type_EC",
plots=c("vlnplot", "jitter"), split.nrow =1 )
dev.off()

## CM markers

In [ ]:
options(repr.plot.width=10, repr.plot.height=8)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/figures/CM_cells_markers_dot_plot_new_24072023.pdf"),
    width = 12,
    height = 8
    )
DotPlot(object = EnC_CM_BAcells, features = c("alcama", "bmpr2b", "cd9a", "col1a1a", "col1a2", "desma", "doc2b", "ednraa", "esama", "fn1a", "hapln1b", "mybpc3", "notch1b", "nrp2a", "nrp2b", "piezo1", "piezo2a.2", "postnb", "ptpn13", "robo1", "tgfb2", "zfpm1", "zfpm2b"), cols=c(  "blue", "red"))+ RotatedAxis()
dev.off()

In [ ]:
options(repr.plot.width=10, repr.plot.height=8)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/figures/CM_EnC_cells_markers_dot_plot_new_24072023.pdf"),
    width = 12,
    height = 8
    )
DotPlot(object = EnC_CM_BAcells, features = c("elnb", "mylka", "nrp1a", "tgfb2", "fn1a", "notch1b", "ednraa", "smad6b", "alcama", "cdh5", "col1a1a", "col5a1", "dchs1b", "hand2", "has2", "klf2a", "spp1", "thbs1a", "col1a1b", "postnb", "rtn4a", "ryr1b", "sdc2", "tie1", "vcana", "col1a2", "crip2", "postna", "slit3", "zfpm1", "zfpm2b", "doc2b", "egfl7", "esama", "hapln1b", "id2b", "bmpr2b", "cd9a", "desma", "mybpc3", "nrp2a", "nrp2b", "piezo1", "piezo2a.2", "ptpn13", "robo1"), cols=c(  "blue", "red"))+ RotatedAxis()
dev.off()

In [ ]:
options(repr.plot.width=12, repr.plot.height=18)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/figures/CM_EnC_cells_markers_violin_plot_new_24072023.pdf"),
    width = 12,
    height = 18
    )
VlnPlot(object = EnC_CM_BAcells, features = c("elnb", "mylka", "nrp1a", "tgfb2", "fn1a", "notch1b", "ednraa", "smad6b", "alcama", "cdh5", "col1a1a", "col5a1", "dchs1b", "hand2", "has2", "klf2a", "spp1", "thbs1a", "col1a1b", "postnb", "rtn4a", "ryr1b", "sdc2", "tie1", "vcana", "col1a2", "crip2", "postna", "slit3", "zfpm1", "zfpm2b", "doc2b", "egfl7", "esama", "hapln1b", "id2b", "bmpr2b", "cd9a", "desma", "mybpc3", "nrp2a", "nrp2b", "piezo1", "piezo2a.2", "ptpn13", "robo1"), stack = TRUE, flip = TRUE, add.noise = FALSE, same.y.lims = TRUE)+NoLegend()
dev.off()

## All cell makers

In [ ]:
Idents(combined_sct)  <-  factor(combined_sct@meta.data$cell_type_EC, levels= sort(unique(combined_sct@meta.data$cell_type_EC), decreasing = TRUE))
Idents(combined_sct) %>% head

In [ ]:
options(repr.plot.width=18, repr.plot.height=12)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/figures/All_cells_markers_dot_plot_magma_25072023_alphabetical_reversed.pdf"),
    width = 18,
    height = 12
    )
DotPlot(object = combined_sct, features = c("col11a2", "col2a1a", "col9a1a","myh6", "smtnl1", "rgs6", "myh7", "slc25a5", "rpl30", "cacna1g", "sorbs1", "mybpc3", "adcy2b", "vwf", "grb10a", "nrg2a", "ABR", "f8", "tbx2a", "smad6b", "abi3bpb", "uqcc2", "tpst1l", "fam169aa", "rspo2", "nrg1", "notch1b", "tbx18", "ebf3a.1", "podxl", "cyt1", "krt4", "tnks1bp1", "hbbe1.3", "hbae3", "hbae1.1", "auts2a", "col1a1b", "ebf3a", "kpna2", "mki67", "aspm", "tnnt3b", "neb", "myhb", "elnb", "lsp1a", "fbln5", "rbfox1", "pbx3b", "GALNTL6", "acsbg1", "ankrd33ba", "slc25a26"), dot.min=0, dot.scale = 10, scale.min = 0, scale.max = 100)+ scale_color_viridis(option = "magma",direction = -1)+ RotatedAxis()
dev.off()

In [ ]:
CM_cells <- subset(combined_sct, subset= (cell_type_EC%in% c( "CM_Atrium","CM_AV_canal","CM_Ventricle")))
# CM_cells
CM_cells@meta.data$cell_type_EC %>% unique
Idents(CM_cells) %>% unique
Idents(CM_cells)  <-  factor(CM_cells@meta.data$cell_type_EC, levels= sort(unique(CM_cells@meta.data$cell_type_EC, decreasing=TRUE)))
Idents(CM_cells)  <-  factor(CM_cells@meta.data$cell_type_EC, levels= sort(unique(CM_cells@meta.data$cell_type_EC), decreasing = TRUE))
Idents(CM_cells) %>% head

In [ ]:
options(repr.plot.width=10, repr.plot.height=8)
dev.copy(
    pdf,
    file = paste0("results_seurat/cell_assigned/figures/CM_cells_markers_dot_plot_new_magma_25072023_alphabetical_reversed.pdf"),
    width = 10,
    height = 8
    )
DotPlot(object = CM_cells, features = c("ptpn13", "mybpc3", "zfpm1", "alcama", "nrp2b", "zfpm2b", "cd9a", "piezo2a.2", "esama", "nrp2a", "bmpr2b", "postnb", "col1a1a", "robo1", "hapln1b", "notch1b", "ednraa", "col1a2", "fn1a", "doc2b", "tgfb2", "desma", "piezo1"),dot.min=0, dot.scale = 10, scale.min = 0, scale.max = 75)+ scale_colour_viridis(option="magma", direction = -1) + RotatedAxis()
dev.off()

## Cell Number Graph

In [ ]:
cell_type_cond_numbers <- table(combined_sct@meta.data$cell_type_EC, combined_sct@meta.data$condition)
cell_type_cond_numbers <- as.data.frame(cell_type_cond_numbers)
cell_type_cond_numbers <- cell_type_cond_numbers %>% dplyr::rename(cell_type = Var1, sample_name = Var2)
cell_type_cond_numbers$sample_name <- factor(cell_type_cond_numbers$sample_name, levels = c("sib","mut"))
# cell_type_cond_numbers %>% write.csv("results_seurat/cell_type_cond_numbers_neuronal_prog.csv")
cell_type_cond_numbers

In [ ]:
# calculate the percentage of cell in each cell type according to condition
cell_type_percentages_cond <- prop.table(table(combined_sct@meta.data$cell_type_EC, combined_sct@meta.data$condition))*100
cell_type_percentages_cond <- as.data.frame(cell_type_percentages_cond)
cell_type_percentages_cond %>% head
cell_type_percentages_cond <- cell_type_percentages_cond %>% dplyr::rename(cell_type = Var1, condition = Var2)
cell_type_percentages_cond$condition <- factor(cell_type_percentages_cond$condition, levels = c("sib","mut"))

In [ ]:
# calculate the percentage of cell in each cell type according to condition
cell_type_percentages_cond <- prop.table(table(combined_sct@meta.data$cell_type_EC, combined_sct@meta.data$condition))*100
cell_type_percentages_cond <- as.data.frame(cell_type_percentages_cond)
cell_type_percentages_cond %>% head
cell_type_percentages_cond <- cell_type_percentages_cond %>% dplyr::rename(cell_type = Var1, condition = Var2)
cell_type_percentages_cond$condition <- factor(cell_type_percentages_cond$condition, levels = c("sib","mut"))
cell_type_percentages_cond %>% write.csv("results_seurat/cell_type_percentages_cond_neuronal_prog.csv")
cell_type_percentages_cond %>% head
# plot these percentages using ggplot2
# save the graph as a pdf file
dev.copy(pdf, "results_seurat/cell_type_percentages_cond_normalized_neuronal_prog_pattern_flipped.pdf", width = 10, height = 10)
# ggplot(cell_type_percentages_cond, aes(x = cell_type, y = Freq, fill = condition)) +
#   geom_bar(stat = "identity", position = "fill") +
#   theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.5)) +
#   labs(x = "Condition", y = "Percentage of cells", fill = "Cell type") 
ggplot(cell_type_percentages_cond, aes(x = cell_type, y = Freq, fill = cell_type, pattern=condition)) +
  scale_colour_paletteer_d("ggthemes::Tableau_20")+
  scale_fill_paletteer_d("ggthemes::Tableau_20")+
  geom_bar_pattern(
  # aes(pattern_colour= cell_type),
  colour= '#5e5d5d',
  pattern_fill= '#4d4d4d',
  pattern_alpha=0.2,
  pattern_angle= 0,
  stat = "identity", position = "fill",   pattern_spacing= 0.01) +
  scale_pattern_manual(values=c('none', 'crosshatch')) +
  theme_bw()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.5)) +
  labs(x = "Condition", y = "Percentage of cells", fill = "Cell type")+coord_flip()
  dev.off()

In [ ]:
# calculate the percentage of cell in each cell type according to condition
# plot these percentages using ggplot2
# save the graph as a pdf file
dev.copy(pdf, "results_seurat/cell_type_percentages_cond_neuronal_prog_pattern_flipped.pdf", width = 10, height = 10)

ggplot(cell_type_percentages_cond, aes(x = cell_type, y = Freq, fill = cell_type, pattern=condition)) +
  scale_colour_paletteer_d("ggthemes::Tableau_20")+
  scale_fill_paletteer_d("ggthemes::Tableau_20")+
  geom_bar_pattern(
  # aes(pattern_colour= cell_type),
  colour= '#5e5d5d',
  pattern_fill= '#4d4d4d',
  pattern_alpha=0.2,
  pattern_angle= 0,
  stat = "identity", position = "dodge",   pattern_spacing= 0.01) +
  scale_pattern_manual(values=c('none', 'crosshatch')) +
  theme_bw()+
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.5)) +
  labs(x = "Condition", y = "Percentage of cells", fill = "Cell type")+coord_flip()
  dev.off()

## Sankey diagram for pathways

In [ ]:
GSE_pathways_files <- list.files(path = "results_seurat/cell_assigned/pathways_DEG_pval05_GSE/Cellular_component", pattern = "*.csv", full.names = TRUE)
GSE_pathways_files

In [ ]:
GSE_names <- gsub("results_seurat/cell_assigned/pathways_DEG_pval05_GSE/Cellular_component/", "", GSE_pathways_files)
GSE_names <- gsub("_gse_GO_CC_pathways_pval05.csv", "", GSE_names)
GSE_names
names(GSE_pathways_files) <- GSE_names
GSE_pathways_files

In [ ]:
pathway_rbind <- purrr::map_df(GSE_pathways_files, 
                              read.csv, .id = 'id')
pathway_rbind$id <- gsub('unknown_2', 'Neuronal_progentiors', pathway_rbind$id)
pathway_rbind$id <- gsub('unknown_1', 'Neurons', pathway_rbind$id)
pathway_rbind %>% head()
pathway_rbind %>% dim()
pathway_rbind %>% distinct(id) 
pathway_rbind %>% distinct(Description)
pathway_rbind %>% distinct(Description) %>% as.character()

In [ ]:
cell_type <- combined_sct@meta.data$cell_type_EC %>% unique
cell_type

In [36]:
lysosome_related_pathways <- c("endosome", "lysosome","lytic vacuole","vacuole","vacuolar membrane", "late endosome", "plasma membrane protein complex"        ) #, "vesicle", "intracellular vesicle", 

In [ ]:
pathway_numbers_selected <-pathway_rbind %>% filter(Description %in% lysosome_related_pathways) 
pathway_numbers_sankey <- pathway_numbers_selected %>% 
add_row(id=cell_type[!(cell_type %in% .$id)]) %>% 
make_long(id, Description)

pathway_numbers_sankey <- pathway_numbers_sankey %>% dplyr::arrange(next_node,node) 

pathway_numbers_sankey$node <- factor(pathway_numbers_sankey$node, levels = c(as.character(cell_type[order(cell_type, decreasing = TRUE)]),lysosome_related_pathways))

pathway_numbers_sankey$x <- gsub("id", "Cell Type", pathway_numbers_sankey$x)
pathway_numbers_sankey$x <- gsub("Description", "Pathway", pathway_numbers_sankey$x)

pathway_numbers_sankey <- pathway_numbers_sankey %>% filter(!is.na(node))

p <- ggplot(pathway_numbers_sankey, aes(x = x, 
               next_x = next_x, 
               node = node, 
               next_node = next_node,
               fill=node,
               label = node
            )
               ) +
  geom_sankey(width=0.05, flow.alpha = .6, type= "sankey") +
   
  theme_sankey(base_size = 32) +
  theme(legend.position = "none")+
NULL
dev.copy(pdf, "results_seurat/cell_assigned/figures/pathway_sankey_graph_labels_gse_resordered_nolablels.pdf", width = 15, height = 15)
print(p)
dev.off()

# Save Rdata and write session info

In [38]:
save.image(file = "Myra_spns_snRNAseq_26052023_mt_removed_seurat_integration_25072023.RData")

In [5]:
load("Myra_spns_snRNAseq_26052023_mt_removed_seurat_integration_25072023.RData")

In [58]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.10.0 
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.10.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_CH.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_CH.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_CH.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_CH.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] viridis_0.6.3               viridisLite_0.4.2          
 [3] liana_0.1.12                future